In [ ]:
# @title ✨ Infinty Time Colab Keeper
import time
import datetime

def keep_colab_alive(duration_hours=24):
    """
    Keeps the Google Colab runtime active by running a loop.

    Args:
        duration_hours (int): The duration in hours for which the script
                              should attempt to keep the runtime alive.
                              Default is 24 hours. Set to 0 for indefinite run
                              (requires manual stop).
    """
    print(f"Script will attempt to keep Colab alive for {duration_hours} hours. (Set to 0 for indefinite run)")
    start_time = datetime.datetime.now()
    end_time = start_time + datetime.timedelta(hours=duration_hours)

    counter = 0
    try:
        while True:
            current_time = datetime.datetime.now()
            if duration_hours > 0 and current_time >= end_time:
                print(f"Maximum duration of {duration_hours} hours reached. Stopping script.")
                break

            counter += 1
            print(f"[{current_time.strftime('%Y-%m-%d %H:%M:%S')}] Colab is alive! Iteration: {counter}")
            # Sleep for a short period to prevent excessive CPU usage
            time.sleep(60) # Sleep for 60 seconds (1 minute)

    except KeyboardInterrupt:
        print("\nScript stopped manually by user (Ctrl+C).")
    except Exception as e:
        print(f"\nAn error occurred: {e}")
    finally:
        print("Colab keep-alive script finished.")

# --- How to use ---
# Call the function to start keeping Colab alive.
# You can specify the duration in hours.
# For example, to keep it alive for 12 hours:
# keep_colab_alive(duration_hours=12)

# To run indefinitely until manually stopped (Ctrl+C):
keep_colab_alive(duration_hours=0) # Set to 0 for indefinite run


In [ ]:
# @title 🧲 TITAN TORRENT - BitTorrent Client for Google Colab
# ==============================================================================
# Ultimate BitTorrent Downloader with Modern UI
# Features: Magnet Links, .torrent Files, Real-time Stats, Google Drive Export
# ==============================================================================

# 1. INSTALLATION
# ==============================================================================
!pip install -q libtorrent ipywidgets tqdm 2>/dev/null || pip install -q python-libtorrent ipywidgets tqdm

import os
import sys
import time
import shutil
import threading
from datetime import datetime, timedelta
from typing import Optional, List, Dict, Callable
from dataclasses import dataclass, field
from enum import Enum, auto

try:
    import libtorrent as lt
except ImportError:
    print("Installing libtorrent...")
    import subprocess
    subprocess.run(['pip', 'install', '-q', 'libtorrent'], check=False)
    import libtorrent as lt

import ipywidgets as widgets
from IPython.display import display, clear_output

# ==============================================================================
# 2. CONFIGURATION
# ==============================================================================
DOWNLOAD_DIR = "/content/torrent_downloads"
TORRENT_RESUME_DIR = "/content/torrent_resume"

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(TORRENT_RESUME_DIR, exist_ok=True)

# ==============================================================================
# 3. DATA CLASSES
# ==============================================================================
class TorrentStatus(Enum):
    QUEUED = auto()
    CHECKING = auto()
    DOWNLOADING_METADATA = auto()
    DOWNLOADING = auto()
    FINISHED = auto()
    SEEDING = auto()
    PAUSED = auto()
    ERROR = auto()

@dataclass
class TorrentStats:
    name: str = ""
    progress: float = 0.0
    download_rate: float = 0.0  # bytes/sec
    upload_rate: float = 0.0
    num_peers: int = 0
    num_seeds: int = 0
    total_size: int = 0
    downloaded: int = 0
    uploaded: int = 0
    eta_seconds: float = 0
    status: TorrentStatus = TorrentStatus.QUEUED
    error_message: str = ""

    @property
    def download_speed_mbps(self) -> float:
        return self.download_rate / (1024 * 1024)

    @property
    def upload_speed_mbps(self) -> float:
        return self.upload_rate / (1024 * 1024)

    @property
    def eta_formatted(self) -> str:
        if self.eta_seconds <= 0 or self.eta_seconds > 86400 * 7:  # > 7 days
            return "--:--"
        mins, secs = divmod(int(self.eta_seconds), 60)
        hours, mins = divmod(mins, 60)
        if hours > 0:
            return f"{hours}:{mins:02d}:{secs:02d}"
        return f"{mins:02d}:{secs:02d}"

    @property
    def progress_percent(self) -> float:
        return self.progress * 100

def format_bytes(size: int) -> str:
    """Format bytes to human readable"""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.2f} {unit}"
        size /= 1024
    return f"{size:.2f} PB"

def format_speed(speed: float) -> str:
    """Format speed in bytes/sec to human readable"""
    if speed < 1024:
        return f"{speed:.0f} B/s"
    elif speed < 1024 * 1024:
        return f"{speed/1024:.1f} KB/s"
    else:
        return f"{speed/(1024*1024):.2f} MB/s"

# ==============================================================================
# 4. TORRENT ENGINE
# ==============================================================================
class TitanTorrentEngine:
    def __init__(self, download_dir: str = DOWNLOAD_DIR):
        self.download_dir = download_dir
        self.session = None
        self.handles: Dict[str, lt.torrent_handle] = {}
        self._running = False
        self._thread = None
        self.log_callback: Callable = print
        self.stats_callback: Callable = None

        self._init_session()

    def _init_session(self):
        """Initialize libtorrent session with optimized settings"""
        # DHT bootstrap nodes are now configured via dht_bootstrap_nodes setting
        settings = {
            'user_agent': 'TitanTorrent/1.0 libtorrent/2.0',
            'listen_interfaces': '0.0.0.0:6881,[::]:6881',
            'download_rate_limit': 0,  # Unlimited
            'upload_rate_limit': 100 * 1024,  # 100 KB/s upload limit
            'active_downloads': 3,
            'active_seeds': 2,
            'active_limit': 5,
            'max_out_request_queue': 500,
            'max_allowed_in_request_queue': 1000,
            'max_peerlist_size': 3000,
            'cache_size': 2048,  # 32 MB cache
            'enable_dht': True,
            'enable_lsd': True,
            'enable_upnp': True,
            'enable_natpmp': True,
            'prefer_rc4': True,
            'announce_to_all_trackers': True,
            'announce_to_all_tiers': True,
            # DHT bootstrap nodes (new API)
            'dht_bootstrap_nodes': 'router.bittorrent.com:6881,router.utorrent.com:6881,dht.transmissionbt.com:6881',
        }

        self.session = lt.session(settings)
        self._log("✅ Torrent engine initialized (DHT enabled)")

    def set_logger(self, callback: Callable):
        self.log_callback = callback

    def set_stats_callback(self, callback: Callable):
        self.stats_callback = callback

    def _log(self, message: str):
        self.log_callback(f"   {message}")

    def add_magnet(self, magnet_link: str) -> Optional[str]:
        """Add torrent from magnet link"""
        try:
            params = lt.parse_magnet_uri(magnet_link)
            params.save_path = self.download_dir

            handle = self.session.add_torrent(params)
            info_hash = str(handle.info_hash())
            self.handles[info_hash] = handle

            self._log(f"🧲 Added magnet: {info_hash[:16]}...")
            return info_hash
        except Exception as e:
            self._log(f"❌ Failed to add magnet: {e}")
            return None

    def add_torrent_file(self, torrent_path: str) -> Optional[str]:
        """Add torrent from .torrent file"""
        try:
            info = lt.torrent_info(torrent_path)
            params = lt.add_torrent_params()
            params.ti = info
            params.save_path = self.download_dir

            handle = self.session.add_torrent(params)
            info_hash = str(handle.info_hash())
            self.handles[info_hash] = handle

            self._log(f"📁 Added torrent: {info.name()}")
            return info_hash
        except Exception as e:
            self._log(f"❌ Failed to add torrent file: {e}")
            return None

    def add_torrent_url(self, url: str) -> Optional[str]:
        """Download .torrent file from URL and add it"""
        try:
            import urllib.request
            import tempfile

            self._log(f"⬇️ Downloading .torrent file...")

            # Download to temp file
            temp_path = os.path.join(TORRENT_RESUME_DIR, "temp.torrent")
            urllib.request.urlretrieve(url, temp_path)

            return self.add_torrent_file(temp_path)
        except Exception as e:
            self._log(f"❌ Failed to download torrent: {e}")
            return None

    def get_stats(self, info_hash: str) -> Optional[TorrentStats]:
        """Get current stats for a torrent"""
        if info_hash not in self.handles:
            return None

        handle = self.handles[info_hash]
        status = handle.status()

        stats = TorrentStats()

        # Get name
        if status.has_metadata:
            stats.name = handle.torrent_file().name()
        else:
            stats.name = f"Fetching metadata... ({status.num_peers} peers)"

        stats.progress = status.progress
        stats.download_rate = status.download_rate
        stats.upload_rate = status.upload_rate
        stats.num_peers = status.num_peers
        stats.num_seeds = status.num_seeds
        stats.total_size = status.total_wanted
        stats.downloaded = status.total_wanted_done
        stats.uploaded = status.total_upload

        # Calculate ETA
        if stats.download_rate > 0:
            remaining = stats.total_size - stats.downloaded
            stats.eta_seconds = remaining / stats.download_rate

        # Map status
        state_map = {
            lt.torrent_status.queued_for_checking: TorrentStatus.QUEUED,
            lt.torrent_status.checking_files: TorrentStatus.CHECKING,
            lt.torrent_status.downloading_metadata: TorrentStatus.DOWNLOADING_METADATA,
            lt.torrent_status.downloading: TorrentStatus.DOWNLOADING,
            lt.torrent_status.finished: TorrentStatus.FINISHED,
            lt.torrent_status.seeding: TorrentStatus.SEEDING,
        }
        stats.status = state_map.get(status.state, TorrentStatus.DOWNLOADING)

        if status.paused:
            stats.status = TorrentStatus.PAUSED

        if status.errc.value() != 0:
            stats.status = TorrentStatus.ERROR
            stats.error_message = status.errc.message()

        return stats

    def get_all_stats(self) -> List[TorrentStats]:
        """Get stats for all torrents"""
        return [self.get_stats(h) for h in self.handles.keys() if self.get_stats(h)]

    def pause(self, info_hash: str):
        if info_hash in self.handles:
            self.handles[info_hash].pause()
            self._log(f"⏸️ Paused: {info_hash[:16]}...")

    def resume(self, info_hash: str):
        if info_hash in self.handles:
            self.handles[info_hash].resume()
            self._log(f"▶️ Resumed: {info_hash[:16]}...")

    def remove(self, info_hash: str, delete_files: bool = False):
        if info_hash in self.handles:
            self.session.remove_torrent(self.handles[info_hash], delete_files)
            del self.handles[info_hash]
            self._log(f"🗑️ Removed: {info_hash[:16]}...")

    def pause_all(self):
        for h in self.handles.values():
            h.pause()
        self._log("⏸️ All torrents paused")

    def resume_all(self):
        for h in self.handles.values():
            h.resume()
        self._log("▶️ All torrents resumed")

    def start_monitoring(self, interval: float = 1.0):
        """Start background monitoring thread"""
        if self._running:
            return

        self._running = True

        def monitor():
            while self._running:
                if self.stats_callback:
                    all_stats = self.get_all_stats()
                    self.stats_callback(all_stats)

                # Check alerts
                alerts = self.session.pop_alerts()
                for alert in alerts:
                    if isinstance(alert, lt.torrent_finished_alert):
                        self._log(f"✅ Download complete: {alert.torrent_name}")
                    elif isinstance(alert, lt.torrent_error_alert):
                        self._log(f"❌ Error: {alert.message()}")

                time.sleep(interval)

        self._thread = threading.Thread(target=monitor, daemon=True)
        self._thread.start()

    def stop_monitoring(self):
        self._running = False
        if self._thread:
            self._thread.join(timeout=2)

    def shutdown(self):
        """Gracefully shutdown the engine"""
        self.stop_monitoring()
        if self.session:
            self.session.pause()
            self._log("🛑 Engine shutdown")

# ==============================================================================
# 5. MODERN UI DASHBOARD
# ==============================================================================
class TitanTorrentDashboard:
    def __init__(self):
        self.engine = TitanTorrentEngine()
        self.current_torrents: Dict[str, TorrentStats] = {}
        self._last_update = time.time()
        self._build_ui()
        self._setup_callbacks()

        # Start monitoring with faster refresh rate for real-time feel
        self.engine.set_stats_callback(self._update_all_stats)
        self.engine.start_monitoring(interval=0.5)  # Update every 0.5 seconds

    def _build_ui(self):
        # CSS Styles
        self.styles = widgets.HTML('''
        <style>
            .torrent-header {
                background: linear-gradient(90deg, #ff6b35, #f7931e);
                -webkit-background-clip: text;
                -webkit-text-fill-color: transparent;
                font-size: 28px;
                font-weight: 800;
                text-align: center;
                margin-bottom: 10px;
            }
            .torrent-subtitle {
                color: #888;
                text-align: center;
                font-size: 12px;
                margin-bottom: 20px;
            }
            .torrent-card {
                background: rgba(255, 107, 53, 0.1);
                border: 1px solid rgba(255, 107, 53, 0.3);
                border-radius: 12px;
                padding: 15px;
                margin: 10px 0;
            }
            .torrent-name {
                font-size: 14px;
                font-weight: bold;
                color: #ff6b35;
                margin-bottom: 8px;
            }
            .stat-row {
                display: flex;
                gap: 20px;
                font-size: 12px;
                color: #aaa;
            }
            .stat-item {
                background: rgba(255,255,255,0.05);
                padding: 8px 12px;
                border-radius: 8px;
            }
            .stat-value {
                font-size: 18px;
                font-weight: bold;
                color: #ff6b35;
            }
            .stat-label {
                font-size: 10px;
                color: #666;
                text-transform: uppercase;
            }
        </style>
        ''')

        # Header
        self.header = widgets.HTML('''
            <div class="torrent-header">🧲 TITAN TORRENT</div>
            <div class="torrent-subtitle">BitTorrent Client for Google Colab</div>
        ''')

        # Input area
        self.input_area = widgets.Textarea(
            placeholder='📋 Paste magnet links or .torrent URLs here (one per line)\n\nExamples:\nmagnet:?xt=urn:btih:...\nhttps://example.com/file.torrent',
            layout=widgets.Layout(width='100%', height='100px')
        )

        # File upload for local .torrent files
        self.file_upload = widgets.FileUpload(
            accept='.torrent',
            multiple=True,
            description='📁 Upload .torrent',
            layout=widgets.Layout(width='100%')
        )
        self.upload_label = widgets.HTML('<div style="color:#888;font-size:12px;margin:5px 0;">หรือ เลือกไฟล์ .torrent จากเครื่อง:</div>')

        # Stats Dashboard
        self.stat_download = widgets.HTML('<div class="stat-item"><div class="stat-value">0 MB/s</div><div class="stat-label">Download</div></div>')
        self.stat_upload = widgets.HTML('<div class="stat-item"><div class="stat-value">0 KB/s</div><div class="stat-label">Upload</div></div>')
        self.stat_peers = widgets.HTML('<div class="stat-item"><div class="stat-value">0</div><div class="stat-label">Peers</div></div>')
        self.stat_active = widgets.HTML('<div class="stat-item"><div class="stat-value">0</div><div class="stat-label">Active</div></div>')

        self.stats_row = widgets.HBox([
            self.stat_download, self.stat_upload, self.stat_peers, self.stat_active
        ], layout=widgets.Layout(justify_content='space-around', margin='10px 0'))

        # Control buttons
        self.btn_add = widgets.Button(
            description='➕ ADD',
            button_style='success',
            layout=widgets.Layout(width='100px', height='40px')
        )
        self.btn_upload = widgets.Button(
            description='📁 UPLOAD',
            button_style='info',
            layout=widgets.Layout(width='100px', height='40px')
        )
        self.btn_pause_all = widgets.Button(
            description='⏸️ PAUSE ALL',
            button_style='warning',
            layout=widgets.Layout(width='110px', height='40px')
        )
        self.btn_resume_all = widgets.Button(
            description='▶️ RESUME',
            button_style='info',
            layout=widgets.Layout(width='100px', height='40px')
        )
        self.btn_clear = widgets.Button(
            description='🗑️ CLEAR',
            button_style='danger',
            layout=widgets.Layout(width='90px', height='40px')
        )

        self.controls = widgets.HBox([
            self.btn_add, self.btn_upload, self.btn_pause_all, self.btn_resume_all, self.btn_clear
        ], layout=widgets.Layout(justify_content='center', gap='8px', margin='15px 0'))

        # Torrent list area
        self.torrent_list = widgets.VBox([], layout=widgets.Layout(
            width='100%',
            max_height='300px',
            overflow_y='auto'
        ))

        # Log output
        self.log_output = widgets.Output(layout=widgets.Layout(
            height='150px',
            overflow_y='auto',
            border='1px solid rgba(255, 107, 53, 0.2)',
            border_radius='8px',
            padding='10px',
            background_color='#0a0a0f'
        ))

        # Footer buttons
        self.btn_export_drive = widgets.Button(
            description='💾 Export to Drive',
            layout=widgets.Layout(width='140px')
        )
        self.btn_clear_log = widgets.Button(
            description='🗑️ Clear Log',
            layout=widgets.Layout(width='100px')
        )

        self.footer = widgets.HBox([
            self.btn_export_drive, self.btn_clear_log
        ], layout=widgets.Layout(justify_content='flex-end', gap='10px', margin='10px 0'))

        # Assemble UI
        self.container = widgets.VBox([
            self.styles,
            self.header,
            self.input_area,
            self.upload_label,
            self.file_upload,
            self.stats_row,
            self.controls,
            widgets.HTML('<div style="color:#ff6b35;font-size:14px;margin:10px 0;">📥 Active Torrents</div>'),
            self.torrent_list,
            widgets.HTML('<div style="color:#ff6b35;font-size:14px;margin:10px 0;">📋 Logs</div>'),
            self.log_output,
            self.footer
        ], layout=widgets.Layout(
            padding='20px',
            border='2px solid rgba(255, 107, 53, 0.3)',
            border_radius='16px',
            width='800px',
            background='linear-gradient(135deg, #0a0a0f 0%, #1a1a2e 100%)'
        ))

    def _setup_callbacks(self):
        self.btn_add.on_click(self._on_add)
        self.btn_upload.on_click(self._on_upload_local)
        self.btn_pause_all.on_click(self._on_pause_all)
        self.btn_resume_all.on_click(self._on_resume_all)
        self.btn_clear.on_click(self._on_clear_all)
        self.btn_clear_log.on_click(self._on_clear_log)
        self.btn_export_drive.on_click(self._on_export_drive)

        self.engine.set_logger(self._log)

    def _log(self, message: str):
        with self.log_output:
            timestamp = datetime.now().strftime("%H:%M:%S")
            print(f"[{timestamp}] {message}")

    def _update_all_stats(self, all_stats: List[TorrentStats]):
        """Update dashboard with all torrent stats - REAL-TIME"""
        # Get current time for live display
        current_time = datetime.now().strftime("%H:%M:%S")

        if not all_stats:
            # Show empty state with live clock
            self.stat_download.value = f'<div class="stat-item"><div class="stat-value">0 B/s</div><div class="stat-label">Download</div></div>'
            self.stat_upload.value = f'<div class="stat-item"><div class="stat-value">0 B/s</div><div class="stat-label">Upload</div></div>'
            self.stat_peers.value = f'<div class="stat-item"><div class="stat-value">0</div><div class="stat-label">Peers</div></div>'
            self.stat_active.value = f'<div class="stat-item"><div class="stat-value">🕐 {current_time}</div><div class="stat-label">Live</div></div>'
            return

        # Aggregate stats
        total_download = sum(s.download_rate for s in all_stats)
        total_upload = sum(s.upload_rate for s in all_stats)
        total_peers = sum(s.num_peers for s in all_stats)
        active_count = len([s for s in all_stats if s.status == TorrentStatus.DOWNLOADING])
        total_downloaded = sum(s.downloaded for s in all_stats)

        # Update stats with live indicator
        pulse = "🟢" if total_download > 0 else "⚪"
        self.stat_download.value = f'<div class="stat-item"><div class="stat-value">{pulse} {format_speed(total_download)}</div><div class="stat-label">Download</div></div>'
        self.stat_upload.value = f'<div class="stat-item"><div class="stat-value">{format_speed(total_upload)}</div><div class="stat-label">Upload</div></div>'
        self.stat_peers.value = f'<div class="stat-item"><div class="stat-value">{total_peers}</div><div class="stat-label">Peers</div></div>'
        self.stat_active.value = f'<div class="stat-item"><div class="stat-value">{active_count}/{len(all_stats)} 🕐{current_time}</div><div class="stat-label">Active • Live</div></div>'

        # Update torrent list with enhanced real-time display
        cards = []
        for stats in all_stats:
            # Animated status icons
            if stats.status == TorrentStatus.DOWNLOADING:
                status_icon = "⬇️" if int(time.time() * 2) % 2 == 0 else "📥"
            elif stats.status == TorrentStatus.DOWNLOADING_METADATA:
                status_icon = "📡" if int(time.time() * 2) % 2 == 0 else "🔍"
            else:
                status_icon = {
                    TorrentStatus.SEEDING: "⬆️",
                    TorrentStatus.FINISHED: "✅",
                    TorrentStatus.PAUSED: "⏸️",
                    TorrentStatus.CHECKING: "�",
                    TorrentStatus.ERROR: "❌",
                }.get(stats.status, "⏳")

            # Enhanced progress bar with glow effect for active downloads
            glow = "box-shadow: 0 0 10px #ff6b35;" if stats.status == TorrentStatus.DOWNLOADING else ""
            progress_bar = f'<div style="background:#333;height:10px;border-radius:5px;overflow:hidden;margin:5px 0;{glow}"><div style="background:linear-gradient(90deg,#ff6b35,#f7931e);height:100%;width:{stats.progress_percent:.1f}%;transition:width 0.3s ease;"></div></div>'

            # Speed indicator with trend
            speed_color = "#00ff88" if stats.download_rate > 500000 else "#ff6b35" if stats.download_rate > 0 else "#666"

            card_html = f'''
            <div class="torrent-card" style="border-color: {'rgba(0,255,136,0.5)' if stats.status == TorrentStatus.DOWNLOADING else 'rgba(255,107,53,0.3)'};">
                <div class="torrent-name">{status_icon} {stats.name[:55]}{'...' if len(stats.name) > 55 else ''}</div>
                {progress_bar}
                <div class="stat-row" style="flex-wrap:wrap;">
                    <span style="font-weight:bold;color:#ff6b35;">📊 {stats.progress_percent:.2f}%</span>
                    <span style="color:{speed_color};">⬇️ {format_speed(stats.download_rate)}</span>
                    <span>⬆️ {format_speed(stats.upload_rate)}</span>
                    <span>👥 {stats.num_peers}</span>
                    <span>⏱️ {stats.eta_formatted}</span>
                    <span>📦 {format_bytes(stats.downloaded)}/{format_bytes(stats.total_size)}</span>
                </div>
            </div>
            '''
            cards.append(widgets.HTML(card_html))

        self.torrent_list.children = tuple(cards) if cards else (widgets.HTML('<div style="color:#666;text-align:center;padding:20px;">No active torrents</div>'),)

    def _on_add(self, b):
        content = self.input_area.value.strip()
        if not content:
            self._log("⚠️ Please enter magnet links or torrent URLs")
            return

        lines = content.strip().split('\n')
        for line in lines:
            line = line.strip()
            if not line:
                continue

            if line.startswith('magnet:'):
                self.engine.add_magnet(line)
            elif line.endswith('.torrent') or 'torrent' in line.lower():
                if line.startswith('http'):
                    self.engine.add_torrent_url(line)
                else:
                    self.engine.add_torrent_file(line)
            else:
                self._log(f"⚠️ Unknown format: {line[:50]}...")

        self.input_area.value = ""

    def _on_upload_local(self, b):
        """Handle local .torrent file uploads"""
        if not self.file_upload.value:
            self._log("⚠️ กรุณาเลือกไฟล์ .torrent ก่อนกด Upload")
            return

        uploaded_files = self.file_upload.value
        count = 0

        # Handle different ipywidgets FileUpload API versions
        # New API: value is dict {filename: {'content': bytes, 'metadata': {...}}}
        # Old API: value is tuple of dicts [{'name': str, 'content': bytes}, ...]
        if isinstance(uploaded_files, dict):
            # New ipywidgets API (8.x+)
            items = [(fname, fdata['content']) for fname, fdata in uploaded_files.items()]
        else:
            # Old ipywidgets API
            items = []
            for file_info in uploaded_files:
                if isinstance(file_info, dict):
                    items.append((file_info['name'], file_info['content']))
                else:
                    items.append((file_info.name, file_info.content))

        for filename, content in items:
            try:

                # Save to temp directory
                temp_path = os.path.join(TORRENT_RESUME_DIR, filename)
                with open(temp_path, 'wb') as f:
                    f.write(content)

                # Add to engine
                result = self.engine.add_torrent_file(temp_path)
                if result:
                    count += 1
                    self._log(f"📁 Loaded: {filename}")
            except Exception as e:
                self._log(f"❌ Error loading file: {e}")

        if count > 0:
            self._log(f"✅ Added {count} torrent(s) from local files")

        # Clear the upload widget by replacing it (FileUpload.value is read-only)
        new_upload = widgets.FileUpload(
            accept='.torrent',
            multiple=True,
            description='📁 Upload .torrent',
            layout=widgets.Layout(width='100%')
        )
        # Replace in container
        container_children = list(self.container.children)
        for i, child in enumerate(container_children):
            if child is self.file_upload:
                container_children[i] = new_upload
                break
        self.container.children = tuple(container_children)
        self.file_upload = new_upload

    def _on_pause_all(self, b):
        self.engine.pause_all()

    def _on_resume_all(self, b):
        self.engine.resume_all()

    def _on_clear_all(self, b):
        for info_hash in list(self.engine.handles.keys()):
            self.engine.remove(info_hash, delete_files=False)
        self._log("🗑️ All torrents removed")

    def _on_clear_log(self, b):
        self.log_output.clear_output()
        self._log("🗑️ Log cleared")

    def _on_export_drive(self, b):
        try:
            from google.colab import drive
            drive.mount('/content/drive', force_remount=False)

            drive_folder = '/content/drive/MyDrive/TitanTorrent'
            os.makedirs(drive_folder, exist_ok=True)

            count = 0
            for item in os.listdir(DOWNLOAD_DIR):
                src = os.path.join(DOWNLOAD_DIR, item)
                dst = os.path.join(drive_folder, item)
                if os.path.isfile(src):
                    shutil.copy2(src, dst)
                    count += 1
                elif os.path.isdir(src):
                    if os.path.exists(dst):
                        shutil.rmtree(dst)
                    shutil.copytree(src, dst)
                    count += 1

            self._log(f"💾 Exported {count} item(s) to Google Drive: /MyDrive/TitanTorrent/")
        except Exception as e:
            self._log(f"⚠️ Drive export error: {e}")

    def _do_refresh(self):
        """Perform a single refresh cycle"""
        try:
            all_stats = self.engine.get_all_stats()
            self._update_all_stats(all_stats)

            # Check alerts
            if self.engine.session:
                alerts = self.engine.session.pop_alerts()
                for alert in alerts:
                    if isinstance(alert, lt.torrent_finished_alert):
                        self._log(f"✅ Download complete: {alert.torrent_name}")
                    elif isinstance(alert, lt.torrent_error_alert):
                        self._log(f"❌ Error: {alert.message()}")
        except Exception as e:
            pass

    def show(self):
        display(self.container)
        self._log("🧲 Titan Torrent พร้อมใช้งาน!")
        self._log("💡 เพิ่ม magnet link หรือ .torrent แล้วกด ADD")
        self._log("🔄 กด REFRESH เพื่ออัปเดตสถานะ")

# ==============================================================================
# 6. KEEP COLAB ALIVE + AUTO REFRESH
# ==============================================================================
dashboard = TitanTorrentDashboard()

# Session timer
session_start = datetime.now()

# Control widgets
refresh_btn = widgets.Button(
    description='🔄 REFRESH',
    button_style='primary',
    layout=widgets.Layout(width='100px', height='35px')
)

keep_alive_toggle = widgets.ToggleButton(
    value=True,  # Default ON
    description='🔥 Keep Alive',
    button_style='success',
    tooltip='Keep Colab session alive',
    layout=widgets.Layout(width='130px')
)

auto_refresh_toggle = widgets.ToggleButton(
    value=True,  # Default ON
    description='⏱️ Auto-Refresh',
    button_style='info',
    layout=widgets.Layout(width='130px')
)

# Status display
status_output = widgets.HTML(value='<span style="color:#888;">⏳ Starting...</span>')
session_timer = widgets.HTML(value='<span style="color:#666;">Session: 00:00:00</span>')

# Iteration counter
iteration_counter = [0]
last_activity = [time.time()]

def format_duration(seconds):
    """Format seconds to HH:MM:SS"""
    hours, remainder = divmod(int(seconds), 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d}"

def do_refresh(b=None):
    """Perform refresh and keep-alive"""
    iteration_counter[0] += 1
    current_time = datetime.now()

    # Update dashboard
    dashboard._do_refresh()

    # Update session timer
    elapsed = (current_time - session_start).total_seconds()
    session_timer.value = f'<span style="color:#00ff88;font-weight:bold;">🕐 Session: {format_duration(elapsed)}</span>'

    # Update status
    status_text = f'🔄 #{iteration_counter[0]} | {current_time.strftime("%H:%M:%S")}'
    if keep_alive_toggle.value:
        status_text += ' | 🔥 Keeping alive'
    status_output.value = f'<span style="color:#00ff88;">{status_text}</span>'

    last_activity[0] = time.time()

refresh_btn.on_click(do_refresh)

def keep_alive_loop():
    """Combined Keep-Alive and Auto-Refresh loop"""
    print(f"� Keep-Alive started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("💡 Toggle 'Keep Alive' ON to prevent Colab from disconnecting")
    print("=" * 50)

    while True:
        try:
            current_time = datetime.now()

            # Auto-refresh if enabled
            if auto_refresh_toggle.value:
                do_refresh()

            # Keep alive ping (even if auto-refresh is off)
            if keep_alive_toggle.value:
                elapsed = (current_time - session_start).total_seconds()
                # Print to console every 5 minutes as heartbeat
                if iteration_counter[0] % 300 == 0:  # Every 5 minutes (300 seconds)
                    print(f"[{current_time.strftime('%Y-%m-%d %H:%M:%S')}] 🔥 Colab alive! Session: {format_duration(elapsed)} | Iterations: {iteration_counter[0]}")

            # Sleep interval
            time.sleep(1)  # Update every 1 second

        except Exception as e:
            status_output.value = f'<span style="color:#ff6b35;">⚠️ Error: {str(e)[:30]}</span>'
            time.sleep(5)

# Start background thread
keep_alive_thread = threading.Thread(target=keep_alive_loop, daemon=True)
keep_alive_thread.start()

# Control panel
control_panel = widgets.HBox([
    refresh_btn,
    keep_alive_toggle,
    auto_refresh_toggle,
    session_timer,
    status_output
], layout=widgets.Layout(
    margin='10px 0',
    padding='10px',
    gap='10px',
    border='1px solid rgba(0, 255, 136, 0.3)',
    border_radius='8px',
    background='rgba(0, 0, 0, 0.3)'
))

# Info message
info_msg = widgets.HTML('''
<div style="background:rgba(255,107,53,0.1);padding:10px;border-radius:8px;margin:10px 0;">
    <b style="color:#ff6b35;">💡 Tips:</b><br>
    <span style="color:#aaa;font-size:12px;">
    • 🔥 <b>Keep Alive</b> = ป้องกัน Colab หลุด (เปิดไว้ตลอด)<br>
    • ⏱️ <b>Auto-Refresh</b> = อัปเดตสถานะอัตโนมัติทุก 1 วินาที<br>
    • 🔄 <b>REFRESH</b> = อัปเดตสถานะทันที
    </span>
</div>
''')

# Display all
display(info_msg)
display(control_panel)
dashboard.show()

# Initial refresh
do_refresh()
print(f"✅ Titan Torrent ready! Session started at {session_start.strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# @title 🔥 TITAN DOWNLOADER - BEYOND GOD MODE 🔥
# ==============================================================================
# ULTIMATE M3U8/DASH/Direct Video Downloader for Google Colab
# Features: Multi-Protocol, Adaptive Threading, Resume, Real-time Dashboard
# ==============================================================================

# 1. INSTALLATION
# ==============================================================================
!pip install curl_cffi pycryptodome tqdm ipywidgets aiohttp xmltodict -q

import os
import re
import sys
import subprocess
import time
import json
import random
import string
import shutil
import hashlib
import binascii
import sqlite3
import asyncio
import threading
from abc import ABC, abstractmethod
from enum import Enum, auto
from dataclasses import dataclass, field
from typing import Optional, List, Dict, Callable, Any, Tuple
from urllib.parse import urlparse, urljoin, parse_qs
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import deque

# Advanced Networking
from curl_cffi import requests as cffi_requests
from Crypto.Cipher import AES
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# ==============================================================================
# 2. CONFIGURATION & CONSTANTS
# ==============================================================================
DOWNLOAD_DIR = "/content/downloads"
TEMP_DIR = "/content/temp_chunks"
HISTORY_DB = "/content/titan_history.db"
RESUME_FILE = "/content/titan_resume.json"

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs(TEMP_DIR, exist_ok=True)

# Chrome 120 Headers - Ultra Realistic
DEFAULT_HEADERS = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9,th;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Cache-Control': 'no-cache',
    'Pragma': 'no-cache',
    'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium\";v=\"120", "Google Chrome";v="120"',
    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# ==============================================================================
# 3. ENUMS & DATA CLASSES
# ==============================================================================
class DownloadStatus(Enum):
    PENDING = auto()
    ANALYZING = auto()
    DOWNLOADING = auto()
    MERGING = auto()
    COMPLETED = auto()
    FAILED = auto()
    PAUSED = auto()
    CANCELLED = auto()

class Protocol(Enum):
    M3U8 = "HLS"
    DASH = "DASH/MPD"
    DIRECT = "Direct"
    UNKNOWN = "Unknown"

@dataclass
class Quality:
    resolution: str
    bandwidth: int
    url: str
    codecs: Optional[str] = None

    def __str__(self):
        return f"{self.resolution} ({self.bandwidth // 1000}kbps)"

@dataclass
class DownloadStats:
    total_segments: int = 0
    completed_segments: int = 0
    failed_segments: int = 0
    bytes_downloaded: int = 0
    start_time: float = 0
    current_speed: float = 0  # bytes/sec
    eta_seconds: float = 0
    retries: int = 0

    @property
    def progress_percent(self) -> float:
        if self.total_segments == 0:
            return 0
        return (self.completed_segments / self.total_segments) * 100

    @property
    def speed_mbps(self) -> float:
        return self.current_speed / (1024 * 1024)

    @property
    def eta_formatted(self) -> str:
        if self.eta_seconds <= 0:
            return "--:--"
        mins, secs = divmod(int(self.eta_seconds), 60)
        hours, mins = divmod(mins, 60)
        if hours > 0:
            return f"{hours}:{mins:02d}:{secs:02d}"
        return f"{mins:02d}:{secs:02d}"

@dataclass
class DownloadTask:
    url: str
    filename: str
    status: DownloadStatus = DownloadStatus.PENDING
    protocol: Protocol = Protocol.UNKNOWN
    quality: Optional[Quality] = None
    available_qualities: List[Quality] = field(default_factory=list)
    stats: DownloadStats = field(default_factory=DownloadStats)
    error_message: str = ""
    output_path: str = ""
    created_at: datetime = field(default_factory=datetime.now)
    logs: List[str] = field(default_factory=list)

# ==============================================================================
# 4. UTILITIES
# ==============================================================================
def generate_auto_name() -> str:
    """Generate unique filename: Xy123z_20251206"""
    rand = ''.join(random.choices(string.ascii_letters + string.digits, k=6))
    date = datetime.now().strftime("%Y%m%d_%H%M%S")
    return f"{rand}_{date}"

def clean_temp(folder: str):
    """Remove temporary folder"""
    if os.path.exists(folder):
        shutil.rmtree(folder)

def format_bytes(size: int) -> str:
    """Format bytes to human readable"""
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024:
            return f"{size:.2f} {unit}"
        size /= 1024
    return f"{size:.2f} PB"

def detect_protocol(url: str) -> Protocol:
    """Auto-detect download protocol from URL"""
    url_lower = url.lower()
    # Remove query params for extension detection
    path = urlparse(url_lower).path

    if '.m3u8' in url_lower or 'hls' in url_lower:
        return Protocol.M3U8
    elif '.mpd' in url_lower or 'dash' in url_lower:
        return Protocol.DASH
    else:
        # Check for any file extension (video, audio, archive, etc.)
        common_extensions = [
            # Video
            '.mp4', '.mkv', '.avi', '.webm', '.mov', '.flv', '.wmv', '.m4v',
            # Audio
            '.mp3', '.wav', '.flac', '.aac', '.ogg', '.m4a', '.wma',
            # Archives
            '.zip', '.rar', '.7z', '.tar', '.gz', '.bz2', '.xz', '.tar.gz', '.tgz',
            # Documents
            '.pdf', '.doc', '.docx', '.xls', '.xlsx', '.ppt', '.pptx',
            # Executables/Installers
            '.exe', '.msi', '.dmg', '.pkg', '.deb', '.rpm', '.appimage',
            # Images
            '.jpg', '.jpeg', '.png', '.gif', '.bmp', '.svg', '.ico', '.webp',
            # Other
            '.iso', '.img', '.bin', '.apk', '.ipa'
        ]
        if any(path.endswith(ext) for ext in common_extensions):
            return Protocol.DIRECT
        # Also check if URL has file-like ending
        if '.' in path.split('/')[-1]:
            return Protocol.DIRECT
    return Protocol.UNKNOWN

def get_url_hash(url: str) -> str:
    """Get MD5 hash of URL for caching"""
    return hashlib.md5(url.encode()).hexdigest()[:12]

# ==============================================================================
# 5. DOWNLOAD HISTORY (SQLite)
# ==============================================================================
class DownloadHistory:
    def __init__(self, db_path: str = HISTORY_DB):
        self.db_path = db_path
        self._init_db()

    def _init_db(self):
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''
                CREATE TABLE IF NOT EXISTS downloads (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    url TEXT NOT NULL,
                    filename TEXT NOT NULL,
                    output_path TEXT,
                    protocol TEXT,
                    quality TEXT,
                    file_size INTEGER DEFAULT 0,
                    segments_total INTEGER DEFAULT 0,
                    segments_completed INTEGER DEFAULT 0,
                    status TEXT DEFAULT 'pending',
                    error_message TEXT,
                    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                    completed_at TIMESTAMP
                )
            ''')
            conn.commit()

    def add(self, task: DownloadTask) -> int:
        with sqlite3.connect(self.db_path) as conn:
            cursor = conn.execute('''
                INSERT INTO downloads (url, filename, protocol, status)
                VALUES (?, ?, ?, ?)
            ''', (task.url, task.filename, task.protocol.value, task.status.name))
            conn.commit()
            return cursor.lastrowid

    def update(self, task_id: int, task: DownloadTask):
        with sqlite3.connect(self.db_path) as conn:
            conn.execute('''
                UPDATE downloads SET
                    output_path = ?,
                    quality = ?,
                    file_size = ?,
                    segments_total = ?,
                    segments_completed = ?,
                    status = ?,
                    error_message = ?,
                    completed_at = CASE WHEN ? = 'COMPLETED' THEN CURRENT_TIMESTAMP ELSE completed_at END
                WHERE id = ?
            ''', (
                task.output_path,
                str(task.quality) if task.quality else None,
                task.stats.bytes_downloaded,
                task.stats.total_segments,
                task.stats.completed_segments,
                task.status.name,
                task.error_message,
                task.status.name,
                task_id
            ))
            conn.commit()

    def get_recent(self, limit: int = 20) -> List[Dict]:
        with sqlite3.connect(self.db_path) as conn:
            conn.row_factory = sqlite3.Row
            rows = conn.execute('''
                SELECT * FROM downloads ORDER BY created_at DESC LIMIT ?
            ''', (limit,)).fetchall()
            return [dict(row) for row in rows]

    def export_csv(self, filepath: str):
        import csv
        with sqlite3.connect(self.db_path) as conn:
            conn.row_factory = sqlite3.Row
            rows = conn.execute('SELECT * FROM downloads ORDER BY created_at DESC').fetchall()

        with open(filepath, 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=rows[0].keys() if rows else [])
            writer.writeheader()
            for row in rows:
                writer.writerow(dict(row))

# ==============================================================================
# 6. RESUME MANAGER
# ==============================================================================
class ResumeManager:
    def __init__(self, resume_file: str = RESUME_FILE):
        self.resume_file = resume_file
        self.data = self._load()

    def _load(self) -> Dict:
        if os.path.exists(self.resume_file):
            try:
                with open(self.resume_file, 'r') as f:
                    return json.load(f)
            except:
                pass
        return {}

    def _save(self):
        with open(self.resume_file, 'w') as f:
            json.dump(self.data, f, indent=2)

    def save_progress(self, url_hash: str, segments_completed: List[int], total: int, temp_folder: str):
        self.data[url_hash] = {
            'completed': segments_completed,
            'total': total,
            'temp_folder': temp_folder,
            'timestamp': datetime.now().isoformat()
        }
        self._save()

    def get_progress(self, url_hash: str) -> Optional[Dict]:
        return self.data.get(url_hash)

    def clear(self, url_hash: str):
        if url_hash in self.data:
            del self.data[url_hash]
            self._save()

# ==============================================================================
# 7. CIRCUIT BREAKER (For Retry Logic)
# ==============================================================================
class CircuitBreaker:
    def __init__(self, failure_threshold: int = 5, recovery_timeout: int = 30):
        self.failure_threshold = failure_threshold
        self.recovery_timeout = recovery_timeout
        self.failures = 0
        self.last_failure_time = 0
        self.state = "CLOSED"  # CLOSED, OPEN, HALF-OPEN

    def record_failure(self):
        self.failures += 1
        self.last_failure_time = time.time()
        if self.failures >= self.failure_threshold:
            self.state = "OPEN"

    def record_success(self):
        self.failures = 0
        self.state = "CLOSED"

    def can_proceed(self) -> bool:
        if self.state == "CLOSED":
            return True
        if self.state == "OPEN":
            if time.time() - self.last_failure_time >= self.recovery_timeout:
                self.state = "HALF-OPEN"
                return True
            return False
        return True  # HALF-OPEN

# ==============================================================================
# 8. PROTOCOL HANDLERS (Abstract Base)
# ==============================================================================
class ProtocolHandler(ABC):
    def __init__(self, session, task: DownloadTask, logger: Callable):
        self.session = session
        self.task = task
        self.log = logger
        self.circuit_breaker = CircuitBreaker()
        self._cancelled = False
        self._paused = False

    @abstractmethod
    def parse(self) -> bool:
        """Parse playlist/manifest and extract segments"""
        pass

    @abstractmethod
    def download(self, progress_callback: Callable) -> bool:
        """Download all segments"""
        pass

    def cancel(self):
        self._cancelled = True

    def pause(self):
        self._paused = True

    def resume(self):
        self._paused = False

    def fetch(self, url: str, desc: str = "data", retries: int = 3, timeout: int = 20):
        """Fetch URL with exponential backoff retry"""
        delay = 1
        for attempt in range(retries):
            if self._cancelled:
                return None

            while self._paused:
                time.sleep(0.5)

            if not self.circuit_breaker.can_proceed():
                self.log(f"⚠️ Circuit breaker OPEN - waiting...")
                time.sleep(self.circuit_breaker.recovery_timeout)
                continue

            try:
                r = self.session.get(url, timeout=timeout, verify=False)
                if r.status_code == 200:
                    self.circuit_breaker.record_success()
                    return r
                elif r.status_code == 403:
                    self.log(f"⚠️ 403 Forbidden: {desc}")
                    self.circuit_breaker.record_failure()
                elif r.status_code == 429:
                    self.log(f"⚠️ Rate limited, waiting {delay}s...")
                    time.sleep(delay)
                    delay = min(delay * 2, 30)
            except Exception as e:
                self.log(f"⚠️ Error fetching {desc}: {e}")
                self.circuit_breaker.record_failure()

            if attempt < retries - 1:
                time.sleep(delay)
                delay = min(delay * 2, 8)  # Exponential backoff, max 8s
                self.task.stats.retries += 1

        return None

# ==============================================================================
# 9. M3U8/HLS HANDLER
# ==============================================================================
class M3U8Handler(ProtocolHandler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.segments: List[str] = []
        self.key_bytes: Optional[bytes] = None
        self.iv_bytes: Optional[bytes] = None
        self.base_url = ""
        self.temp_folder = ""

    def parse(self) -> bool:
        self.log("🔍 Analyzing M3U8 playlist...")
        r = self.fetch(self.task.url, "Master Playlist")
        if not r:
            return False

        content = r.text
        self.base_url = self.task.url

        # Handle Master Playlist
        if "#EXT-X-STREAM-INF" in content:
            self.log("📋 Master playlist detected, extracting qualities...")
            self.task.available_qualities = self._parse_qualities(content)

            if not self.task.available_qualities:
                self.log("❌ No qualities found in master playlist")
                return False

            # Use selected quality or best available
            if self.task.quality:
                selected = self.task.quality
            else:
                selected = max(self.task.available_qualities, key=lambda q: q.bandwidth)
                self.task.quality = selected

            self.log(f"🎯 Selected: {selected}")

            r2 = self.fetch(selected.url, "Media Playlist")
            if not r2:
                return False
            content = r2.text
            self.base_url = selected.url

        # Parse segments and keys
        return self._parse_segments(content)

    def _parse_qualities(self, content: str) -> List[Quality]:
        qualities = []
        lines = content.splitlines()

        for i, line in enumerate(lines):
            if line.startswith("#EXT-X-STREAM-INF"):
                # Extract info
                bw_match = re.search(r'BANDWIDTH=(\d+)', line)
                res_match = re.search(r'RESOLUTION=(\d+x\d+)', line)
                codec_match = re.search(r'CODECS="([^"]+)"', line)

                bandwidth = int(bw_match.group(1)) if bw_match else 0
                resolution = res_match.group(1) if res_match else "unknown"
                codecs = codec_match.group(1) if codec_match else None

                # Get URL
                if i + 1 < len(lines):
                    url_line = lines[i + 1].strip()
                    if url_line and not url_line.startswith("#"):
                        url = urljoin(self.base_url, url_line)

                        # Convert resolution to friendly format
                        if 'x' in resolution:
                            height = resolution.split('x')[1]
                            friendly_res = f"{height}p"
                        else:
                            friendly_res = resolution

                        qualities.append(Quality(
                            resolution=friendly_res,
                            bandwidth=bandwidth,
                            url=url,
                            codecs=codecs
                        ))

        # Sort by bandwidth (highest first)
        return sorted(qualities, key=lambda q: q.bandwidth, reverse=True)

    def _parse_segments(self, content: str) -> bool:
        self.segments = []
        lines = content.splitlines()

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # Parse encryption key
            if line.startswith("#EXT-X-KEY"):
                uri_match = re.search(r'URI="([^"]+)"', line)
                if uri_match:
                    key_url = urljoin(self.base_url, uri_match.group(1))
                    self.log("🔑 Downloading decryption key...")
                    k_res = self.fetch(key_url, "Decryption Key")
                    if k_res:
                        self.key_bytes = k_res.content
                    else:
                        self.log("❌ Failed to get decryption key")
                        return False

                iv_match = re.search(r'IV=0x([0-9a-fA-F]+)', line)
                if iv_match:
                    self.iv_bytes = binascii.unhexlify(iv_match.group(1))

            # Segment URL
            elif not line.startswith("#"):
                seg_url = urljoin(self.base_url, line)
                self.segments.append(seg_url)

        self.task.stats.total_segments = len(self.segments)
        self.log(f"📦 Found {len(self.segments)} segments")
        return len(self.segments) > 0

    def download(self, progress_callback: Callable) -> bool:
        if not self.segments:
            return False

        self.temp_folder = os.path.join(TEMP_DIR, self.task.filename)
        os.makedirs(self.temp_folder, exist_ok=True)

        # Check for resume
        resume_mgr = ResumeManager()
        url_hash = get_url_hash(self.task.url)
        resume_data = resume_mgr.get_progress(url_hash)

        completed_indices = set()
        if resume_data and resume_data.get('temp_folder') == self.temp_folder:
            completed_indices = set(resume_data.get('completed', []))
            self.log(f"🔄 Resuming: {len(completed_indices)}/{len(self.segments)} already done")
            self.task.stats.completed_segments = len(completed_indices)

        # Adaptive thread count
        seg_count = len(self.segments)
        if seg_count < 50:
            max_workers = 8
        elif seg_count < 200:
            max_workers = 16
        elif seg_count < 500:
            max_workers = 24
        else:
            max_workers = 32

        self.log(f"🚀 Downloading with {max_workers} threads...")
        self.task.stats.start_time = time.time()

        # Download segments
        lock = threading.Lock()
        completed_list = list(completed_indices)

        def download_segment(args):
            index, url = args
            if self._cancelled:
                return (index, False)

            while self._paused:
                time.sleep(0.5)

            if index in completed_indices:
                return (index, True)

            retries = 3
            delay = 1

            while retries > 0:
                if self._cancelled:
                    return (index, False)

                try:
                    r = self.session.get(url, timeout=15, verify=False)
                    if r.status_code == 200:
                        data = r.content

                        # Decrypt if encrypted
                        if self.key_bytes:
                            iv = self.iv_bytes if self.iv_bytes else (index).to_bytes(16, 'big')
                            cipher = AES.new(self.key_bytes, AES.MODE_CBC, iv)
                            data = cipher.decrypt(data)
                            # Remove PKCS7 padding
                            if data:
                                pad_len = data[-1]
                                if pad_len < 16:
                                    data = data[:-pad_len]

                        # Write to file
                        fname = os.path.join(self.temp_folder, f"{index:05d}.ts")
                        with open(fname, "wb") as f:
                            f.write(data)

                        with lock:
                            completed_list.append(index)
                            self.task.stats.bytes_downloaded += len(data)

                        return (index, True)
                except Exception:
                    pass

                retries -= 1
                time.sleep(delay)
                delay = min(delay * 2, 4)

            return (index, False)

        # Execute with thread pool
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []
            for i, seg_url in enumerate(self.segments):
                if i not in completed_indices:
                    futures.append(executor.submit(download_segment, (i, seg_url)))

            for future in as_completed(futures):
                if self._cancelled:
                    break

                idx, success = future.result()
                if success:
                    self.task.stats.completed_segments += 1
                else:
                    self.task.stats.failed_segments += 1

                # Update speed and ETA
                elapsed = time.time() - self.task.stats.start_time
                if elapsed > 0:
                    self.task.stats.current_speed = self.task.stats.bytes_downloaded / elapsed
                    remaining = self.task.stats.total_segments - self.task.stats.completed_segments
                    if self.task.stats.completed_segments > 0:
                        avg_time = elapsed / self.task.stats.completed_segments
                        self.task.stats.eta_seconds = remaining * avg_time

                # Save progress for resume
                resume_mgr.save_progress(url_hash, completed_list, len(self.segments), self.temp_folder)

                # Call progress callback
                progress_callback(self.task.stats)

        if self._cancelled:
            return False

        # Verify completion
        success_rate = self.task.stats.completed_segments / len(self.segments)
        if success_rate < 0.8:
            self.log(f"❌ Too many failed segments ({self.task.stats.failed_segments})")
            return False

        # Merge segments
        return self._merge_segments()

    def _merge_segments(self) -> bool:
        self.log("🔄 Merging segments...")

        list_path = os.path.join(self.temp_folder, "list.txt")
        ts_files = sorted([f for f in os.listdir(self.temp_folder) if f.endswith(".ts")])

        with open(list_path, "w") as f:
            for ts in ts_files:
                f.write(f"file '{ts}'\n")

        final_path = os.path.join(DOWNLOAD_DIR, f"{self.task.filename}.mp4")

        # FFmpeg merge
        cmd = f"ffmpeg -y -f concat -safe 0 -i '{list_path}' -c copy -bsf:a aac_adtstoasc '{final_path}' > /dev/null 2>&1"
        result = os.system(cmd)

        if os.path.exists(final_path) and os.path.getsize(final_path) > 0:
            self.task.output_path = final_path
            self.log(f"✅ Saved: {self.task.filename}.mp4")

            # Clean up
            clean_temp(self.temp_folder)
            ResumeManager().clear(get_url_hash(self.task.url))
            return True
        else:
            self.log("❌ Merge failed")
            return False

# ==============================================================================
# 10. DIRECT DOWNLOAD HANDLER
# ==============================================================================
class DirectHandler(ProtocolHandler):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.file_size = 0

    def parse(self) -> bool:
        self.log("🔍 Analyzing direct URL...")
        try:
            r = self.session.head(self.task.url, timeout=10, verify=False)
            if r.status_code == 200:
                self.file_size = int(r.headers.get('Content-Length', 0))
                self.task.stats.total_segments = 1
                self.log(f"📦 File size: {format_bytes(self.file_size)}")
                return True

            # Some servers don't support HEAD
            self.task.stats.total_segments = 1
            return True
        except Exception as e:
            self.log(f"⚠️ Error: {e}")
            return True  # Still try to download

    def download(self, progress_callback: Callable) -> bool:
        self.log("⬇️ Downloading file...")
        self.task.stats.start_time = time.time()

        # Determine extension from URL or Content-Disposition
        parsed = urlparse(self.task.url)
        path = parsed.path

        # Try to get extension from URL path
        ext = ''
        if '.' in path.split('/')[-1]:
            filename_from_url = path.split('/')[-1]
            ext_match = re.search(r'(\.[a-zA-Z0-9]+)$', filename_from_url)
            if ext_match:
                ext = ext_match.group(1).lower()

        # If no extension found, default to .bin
        if not ext:
            ext = '.bin'

        final_path = os.path.join(DOWNLOAD_DIR, f"{self.task.filename}{ext}")

        try:
            r = self.session.get(self.task.url, timeout=120, verify=False, stream=True)
            if r.status_code != 200:
                self.log(f"❌ Download failed: HTTP {r.status_code}")
                return False

            total_size = int(r.headers.get('Content-Length', 0))
            downloaded = 0

            with open(final_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024 * 1024):  # 1MB chunks
                    if self._cancelled:
                        return False

                    while self._paused:
                        time.sleep(0.5)

                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)
                        self.task.stats.bytes_downloaded = downloaded

                        # Update speed
                        elapsed = time.time() - self.task.stats.start_time
                        if elapsed > 0:
                            self.task.stats.current_speed = downloaded / elapsed
                            if total_size > 0:
                                remaining = total_size - downloaded
                                self.task.stats.eta_seconds = remaining / self.task.stats.current_speed

                        progress_callback(self.task.stats)

            self.task.stats.completed_segments = 1
            self.task.output_path = final_path
            self.log(f"✅ Saved: {self.task.filename}{ext}")
            return True

        except Exception as e:
            self.log(f"❌ Download error: {e}")
            return False

# ==============================================================================
# 10.5. POST PROCESSOR (NFO & SCREENSHOT)
# ==============================================================================
class PostProcessor:
    def __init__(self, logger: Callable):
        self.log = logger

    def run(self, task: DownloadTask):
        if not task.output_path or not os.path.exists(task.output_path):
            return

        self.log("🎨 Starting Post-Processing...")

        try:
            # 1. Get Media Info
            info = self._get_media_info(task.output_path)

            # 2. Generate NFO
            self._generate_nfo(task, info)

            # 3. Generate Screenshots
            self._generate_screenshots(task, info)

            self.log("✨ Post-Processing completed!")

        except Exception as e:
            self.log(f"⚠️ Post-Processing error: {e}")

    def _get_media_info(self, filepath: str) -> Dict:
        cmd = f'ffprobe -v quiet -print_format json -show_format -show_streams "{filepath}"'
        try:
            result = subprocess.check_output(cmd, shell=True).decode('utf-8')
            return json.loads(result)
        except Exception:
            return None

    def _generate_nfo(self, task: DownloadTask, info: Dict):
        try:
            filepath = task.output_path
            filename = os.path.basename(filepath)
            filesize = os.path.getsize(filepath) / (1024 * 1024) # MB

            fmt = info.get('format', {}) if info else {}
            duration = float(fmt.get('duration', 0))

            # Find best video/audio streams
            video = next((s for s in info.get('streams', []) if s['codec_type'] == 'video'), {}) if info else {}
            audio = next((s for s in info.get('streams', []) if s['codec_type'] == 'audio'), {}) if info else {}

            # Detailed specs
            width = video.get('width', 'N/A')
            height = video.get('height', 'N/A')
            fps = video.get('r_frame_rate', 'N/A')
            if '/' in fps:
                try:
                    num, den = map(int, fps.split('/'))
                    fps = f"{num/den:.2f}"
                except: pass

            audio_rate = audio.get('sample_rate', 'N/A')
            if audio_rate != 'N/A':
                audio_rate = f"{int(audio_rate)//1000} kHz"

            nfo_content = f"""
████████╗██╗████████╗ █████╗ ███╗   ██╗
╚══██╔══╝██║╚══██╔══╝██╔══██╗████╗  ██║
   ██║   ██║   ██║   ███████║██╔██╗ ██║
   ██║   ██║   ██║   ██╔══██║██║╚██╗██║
   ██║   ██║   ██║   ██║  ██║██║ ╚████║
   ╚═╝   ╚═╝   ╚═╝   ╚═╝  ╚═╝╚═╝  ╚═══╝
   BEYOND GOD MODE • {datetime.now().strftime("%Y")}
================================================================================
 RELEASE INFORMATION
================================================================================
 Release Name...: {filename}
 Generated on...: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
 File Size......: {filesize:.2f} MB
 Duration.......: {int(duration // 60)} min {int(duration % 60)} sec

 VIDEO SPECS
 -------------------------------------------------------------------------------
 Container......: {fmt.get('format_name', 'mp4').upper()}
 Codec..........: {video.get('codec_name', 'unknown').upper()}
 Profile........: {video.get('profile', 'N/A')}
 Resolution.....: {width}x{height}
 Frame Rate.....: {fps} fps
 Bitrate........: {int(video.get('bit_rate', '0')) // 1000 if video.get('bit_rate') else 'N/A'} kbps
 Pixel Format...: {video.get('pix_fmt', 'N/A')}

 AUDIO SPECS
 -------------------------------------------------------------------------------
 Codec..........: {audio.get('codec_name', 'unknown').upper()}
 Channels.......: {audio.get('channels', '2')}
 Sample Rate....: {audio_rate}
 Language.......: {audio.get('tags', {}).get('language', 'N/A').upper()}

 DOWNLOAD LOGS
================================================================================
{chr(10).join(task.logs)}
================================================================================
"""
            nfo_path = os.path.splitext(filepath)[0] + ".nfo"
            with open(nfo_path, "w", encoding="utf-8") as f:
                f.write(nfo_content.strip())

            self.log(f"📝 Generated NFO: {os.path.basename(nfo_path)}")
        except Exception as e:
            self.log(f"❌ NFO Generation failed: {e}")

    def _generate_screenshots(self, task: DownloadTask, info: Dict):
        try:
            filepath = task.output_path
            output_base = os.path.splitext(filepath)[0]

            duration = 0
            if info:
                duration = float(info.get('format', {}).get('duration', 0))

            if duration == 0:
                 # Fallback fetch
                cmd = f'ffprobe -v error -show_entries format=duration -of default=noprint_wrappers=1:nokey=1 "{filepath}"'
                try:
                    duration = float(subprocess.check_output(cmd, shell=True).decode('utf-8').strip())
                except:
                    duration = 0

            if duration <= 0:
                self.log("⚠️ Could not determine duration for screenshots")
                return

            # Contact Sheet Logic (3x3 grid = 9 images)
            interval = duration / 10 # Sample slightly more frequently to avoid end

            self.log("📸 Generating contact sheet...")
            out_name = f"{output_base}_preview.jpg"

            # Tile filter: 3x3 grid, resize to 480px width
            cmd = f'ffmpeg -y -v quiet -i "{filepath}" -vf "select=\'isnan(prev_selected_t)+gte(t-prev_selected_t,{interval})\',scale=480:-1,tile=3x3" -vframes 1 -q:v 2 "{out_name}" > /dev/null 2>&1'
            os.system(cmd)

            if os.path.exists(out_name):
                self.log(f"✅ Generated Contact Sheet: {os.path.basename(out_name)}")
            else:
                self.log("❌ Failed to create contact sheet file")

        except Exception as e:
            self.log(f"❌ Screenshot generation failed: {e}")

# ==============================================================================
# 11. TITAN DOWNLOADER CORE
# ==============================================================================
class TitanCore:
    def __init__(self):
        self.session = cffi_requests.Session(impersonate="chrome120")
        self.session.headers.update(DEFAULT_HEADERS)
        self.history = DownloadHistory()
        self.current_handler: Optional[ProtocolHandler] = None
        self.log_callback: Callable = print
        self.post_processor = PostProcessor(self._log)
        self.active_task: Optional[DownloadTask] = None

    def set_logger(self, callback: Callable):
        self.log_callback = callback

    def _log(self, message: str):
        timestamp = datetime.now().strftime("[%H:%M:%S]")
        full_msg = f"{timestamp} {message}"
        self.log_callback(f"   {message}")

        if self.active_task:
            self.active_task.logs.append(full_msg)

    def process(self, task: DownloadTask, progress_callback: Callable) -> bool:
        self.active_task = task

        # Set referer based on URL
        parsed = urlparse(task.url)
        origin = f"{parsed.scheme}://{parsed.netloc}"
        self.session.headers.update({
            'Origin': origin,
            'Referer': origin + '/'
        })

        # Detect protocol
        task.protocol = detect_protocol(task.url)
        self._log(f"📡 Detected protocol: {task.protocol.value}")

        # Add to history
        history_id = self.history.add(task)

        # Create appropriate handler
        if task.protocol == Protocol.M3U8:
            self.current_handler = M3U8Handler(self.session, task, self._log)
        elif task.protocol == Protocol.DIRECT:
            self.current_handler = DirectHandler(self.session, task, self._log)
        else:
            # Try Direct download as fallback for unknown protocols
            self._log("⚠️ Unknown protocol, trying direct download...")
            self.current_handler = DirectHandler(self.session, task, self._log)

        # Parse
        task.status = DownloadStatus.ANALYZING
        if not self.current_handler.parse():
            task.status = DownloadStatus.FAILED
            task.error_message = "Failed to parse playlist/manifest"
            self.history.update(history_id, task)
            self.active_task = None
            return False

        # Download
        task.status = DownloadStatus.DOWNLOADING
        if not self.current_handler.download(progress_callback):
            if task.status != DownloadStatus.CANCELLED:
                task.status = DownloadStatus.FAILED
                task.error_message = "Download failed"
            self.history.update(history_id, task)
            self.active_task = None
            return False

        # Post-Process
        if task.status != DownloadStatus.CANCELLED:
            self.post_processor.run(task)

        # Complete
        task.status = DownloadStatus.COMPLETED
        self.history.update(history_id, task)
        self.active_task = None
        return True

    def cancel(self):
        if self.current_handler:
            self.current_handler.cancel()

    def pause(self):
        if self.current_handler:
            self.current_handler.pause()

    def resume(self):
        if self.current_handler:
            self.current_handler.resume()

# ==============================================================================
# 12. BATCH QUEUE MANAGER
# ==============================================================================
class BatchQueue:
    def __init__(self, titan_core: TitanCore):
        self.core = titan_core
        self.queue: deque[DownloadTask] = deque()
        self.completed: List[DownloadTask] = []
        self.failed: List[DownloadTask] = []
        self.current_task: Optional[DownloadTask] = None
        self._running = False

    def add(self, url: str, filename: str = None):
        if not filename:
            filename = generate_auto_name()
        task = DownloadTask(url=url, filename=filename)
        self.queue.append(task)
        return task

    def process_all(self, progress_callback: Callable, task_callback: Callable = None):
        self._running = True

        while self.queue and self._running:
            self.current_task = self.queue.popleft()

            if task_callback:
                task_callback(self.current_task, "start")

            success = self.core.process(self.current_task, progress_callback)

            if success:
                self.completed.append(self.current_task)
            else:
                self.failed.append(self.current_task)

            if task_callback:
                task_callback(self.current_task, "end")

        self.current_task = None
        self._running = False

    def stop(self):
        self._running = False
        self.core.cancel()

# ==============================================================================
# 13. MODERN UI DASHBOARD
# ==============================================================================
class TitanDashboard:
    def __init__(self):
        self.titan = TitanCore()
        self.batch_queue = BatchQueue(self.titan)
        self._build_ui()
        self._setup_callbacks()

    def _build_ui(self):
        # CSS Styles - Glassmorphism Dark Theme
        self.styles = widgets.HTML('''
        <style>
            .titan-container {
                background: linear-gradient(135deg, #0a0a0f 0%, #1a1a2e 100%);
                border: 2px solid rgba(0, 255, 136, 0.3);
                border-radius: 16px;
                padding: 24px;
                font-family: 'Segoe UI', system-ui, sans-serif;
                box-shadow: 0 8px 32px rgba(0, 255, 136, 0.1);
            }
            .titan-header {
                background: linear-gradient(90deg, #00ff88, #00ccff);
                -webkit-background-clip: text;
                -webkit-text-fill-color: transparent;
                font-size: 28px;
                font-weight: 800;
                text-align: center;
                margin-bottom: 20px;
                text-shadow: 0 0 30px rgba(0, 255, 136, 0.5);
            }
            .titan-subtitle {
                color: #888;
                text-align: center;
                font-size: 12px;
                margin-top: -15px;
                margin-bottom: 20px;
            }
            .stat-card {
                background: rgba(255, 255, 255, 0.05);
                border: 1px solid rgba(255, 255, 255, 0.1);
                border-radius: 12px;
                padding: 12px 16px;
                margin: 4px;
                backdrop-filter: blur(10px);
            }
            .stat-value {
                font-size: 24px;
                font-weight: bold;
                color: #00ff88;
            }
            .stat-label {
                font-size: 11px;
                color: #666;
                text-transform: uppercase;
            }
            .quality-btn {
                background: rgba(0, 255, 136, 0.1) !important;
                border: 1px solid rgba(0, 255, 136, 0.3) !important;
                color: #00ff88 !important;
                border-radius: 8px !important;
                margin: 2px !important;
                transition: all 0.3s ease !important;
            }
            .quality-btn:hover {
                background: rgba(0, 255, 136, 0.2) !important;
                box-shadow: 0 0 15px rgba(0, 255, 136, 0.3) !important;
            }
            .quality-btn.selected {
                background: #00ff88 !important;
                color: #000 !important;
            }
        </style>
        ''')

        # Header
        self.header = widgets.HTML('''
            <div class="titan-header">🔥 TITAN DOWNLOADER</div>
            <div class="titan-subtitle">BEYOND GOD MODE • M3U8 / DASH / DIRECT</div>
        ''')

        # URL Input
        self.url_input = widgets.Textarea(
            placeholder='📋 Paste URLs here (one per line)\n\nFormat:\nhttps://example.com/video.m3u8\nhttps://example.com/video.m3u8|custom_name\nhttps://example.com/video.mp4',
            layout=widgets.Layout(width='100%', height='150px')
        )
        self.url_input.add_class('titan-input')

        # Custom filename
        self.filename_input = widgets.Text(
            placeholder='📝 Custom filename (optional)',
            layout=widgets.Layout(width='100%')
        )

        # Quality selector (will be populated after URL analysis)
        self.quality_label = widgets.HTML('<b style="color:#888;">Quality:</b>')
        self.quality_dropdown = widgets.Dropdown(
            options=[('Auto (Best)', None)],
            value=None,
            layout=widgets.Layout(width='200px')
        )

        # Stats Dashboard
        self.stat_speed = widgets.HTML('<div class="stat-card"><div class="stat-value">--</div><div class="stat-label">Speed</div></div>')
        self.stat_progress = widgets.HTML('<div class="stat-card"><div class="stat-value">0%</div><div class="stat-label">Progress</div></div>')
        self.stat_eta = widgets.HTML('<div class="stat-card"><div class="stat-value">--:--</div><div class="stat-label">ETA</div></div>')
        self.stat_segments = widgets.HTML('<div class="stat-card"><div class="stat-value">0/0</div><div class="stat-label">Segments</div></div>')

        self.stats_row = widgets.HBox([
            self.stat_speed, self.stat_progress, self.stat_eta, self.stat_segments
        ], layout=widgets.Layout(justify_content='space-around', margin='10px 0'))

        # Progress bar
        self.progress_bar = widgets.FloatProgress(
            value=0, min=0, max=100,
            bar_style='success',
            layout=widgets.Layout(width='100%', height='25px')
        )
        self.progress_label = widgets.HTML('<div style="text-align:center;color:#666;font-size:12px;">Ready</div>')

        # Control buttons
        self.btn_start = widgets.Button(
            description='🚀 START',
            button_style='success',
            layout=widgets.Layout(width='150px', height='45px')
        )
        self.btn_pause = widgets.Button(
            description='⏸️ PAUSE',
            button_style='warning',
            layout=widgets.Layout(width='100px', height='45px'),
            disabled=True
        )
        self.btn_cancel = widgets.Button(
            description='❌ CANCEL',
            button_style='danger',
            layout=widgets.Layout(width='100px', height='45px'),
            disabled=True
        )
        self.btn_analyze = widgets.Button(
            description='🔍 ANALYZE',
            button_style='info',
            layout=widgets.Layout(width='100px', height='45px')
        )

        self.controls = widgets.HBox([
            self.btn_start, self.btn_pause, self.btn_cancel, self.btn_analyze
        ], layout=widgets.Layout(justify_content='center', gap='10px', margin='15px 0'))

        # Log output
        self.log_output = widgets.Output(layout=widgets.Layout(
            height='250px',
            overflow_y='auto',
            border='1px solid rgba(0, 255, 136, 0.2)',
            border_radius='8px',
            padding='10px',
            background_color='#0a0a0f'
        ))

        # History section
        self.btn_show_history = widgets.Button(
            description='📜 History',
            layout=widgets.Layout(width='100px')
        )
        self.btn_export_drive = widgets.Button(
            description='💾 Save to Drive',
            layout=widgets.Layout(width='120px')
        )
        self.btn_clear_log = widgets.Button(
            description='🗑️ Clear Log',
            layout=widgets.Layout(width='100px')
        )

        self.footer = widgets.HBox([
            self.btn_show_history, self.btn_export_drive, self.btn_clear_log
        ], layout=widgets.Layout(justify_content='flex-end', gap='10px', margin='10px 0'))

        # Assemble UI
        self.container = widgets.VBox([
            self.styles,
            self.header,
            self.url_input,
            widgets.HBox([self.filename_input], layout=widgets.Layout(margin='10px 0')),
            widgets.HBox([self.quality_label, self.quality_dropdown], layout=widgets.Layout(align_items='center', gap='10px')),
            self.stats_row,
            self.progress_bar,
            self.progress_label,
            self.controls,
            widgets.HTML('<div style="color:#00ff88;font-size:14px;margin:10px 0;">📋 Logs</div>'),
            self.log_output,
            self.footer
        ], layout=widgets.Layout(
            padding='20px',
            border='2px solid rgba(0, 255, 136, 0.3)',
            border_radius='16px',
            width='800px',
            background='linear-gradient(135deg, #0a0a0f 0%, #1a1a2e 100%)'
        ))

    def _setup_callbacks(self):
        self.btn_start.on_click(self._on_start)
        self.btn_pause.on_click(self._on_pause)
        self.btn_cancel.on_click(self._on_cancel)
        self.btn_analyze.on_click(self._on_analyze)
        self.btn_clear_log.on_click(self._on_clear_log)
        self.btn_show_history.on_click(self._on_show_history)
        self.btn_export_drive.on_click(self._on_export_drive)

        # Set logger
        self.titan.set_logger(self._log)

    def _log(self, message: str):
        with self.log_output:
            timestamp = datetime.now().strftime("%H:%M:%S")
            print(f"[{timestamp}] {message}")

    def _update_stats(self, stats: DownloadStats):
        self.stat_speed.value = f'<div class="stat-card"><div class="stat-value">{stats.speed_mbps:.2f} MB/s</div><div class="stat-label">Speed</div></div>'
        self.stat_progress.value = f'<div class="stat-card"><div class="stat-value">{stats.progress_percent:.1f}%</div><div class="stat-label">Progress</div></div>'
        self.stat_eta.value = f'<div class="stat-card"><div class="stat-value">{stats.eta_formatted}</div><div class="stat-label">ETA</div></div>'
        self.stat_segments.value = f'<div class="stat-card"><div class="stat-value">{stats.completed_segments}/{stats.total_segments}</div><div class="stat-label">Segments</div></div>'
        self.progress_bar.value = stats.progress_percent

    def _on_start(self, b):
        content = self.url_input.value.strip()
        if not content:
            self._log("⚠️ Please enter URL(s)")
            return

        # Disable controls
        self.btn_start.disabled = True
        self.btn_pause.disabled = False
        self.btn_cancel.disabled = False

        self.progress_label.value = '<div style="text-align:center;color:#00ff88;font-size:12px;">🚀 Starting...</div>'

        # Parse URLs
        lines = content.strip().split('\n')
        total = len([l for l in lines if l.strip()])

        self._log(f"🔥 Starting Titan Downloader: {total} task(s)")

        for i, line in enumerate(lines):
            line = line.strip()
            if not line:
                continue

            # Parse URL and optional name
            if '|' in line:
                url, name = line.split('|', 1)
                name = name.strip()
            else:
                url = line
                name = self.filename_input.value.strip() or generate_auto_name()

            url = url.strip()
            self._log(f"▶️ [{i+1}/{total}] Processing: {name}")

            # Create task
            task = DownloadTask(url=url, filename=name)

            # Select quality if specified
            if self.quality_dropdown.value is not None:
                task.quality = self.quality_dropdown.value

            # Process
            success = self.titan.process(task, self._update_stats)

            if success:
                self._log(f"✅ Completed: {name}")
            else:
                self._log(f"❌ Failed: {name} - {task.error_message}")

            self._log("-" * 40)

        self._log("🏁 All tasks completed!")

        # Re-enable controls
        self.btn_start.disabled = False
        self.btn_pause.disabled = True
        self.btn_cancel.disabled = True
        self.progress_label.value = '<div style="text-align:center;color:#00ff88;font-size:12px;">✅ Done!</div>'

    def _on_pause(self, b):
        if self.btn_pause.description == '⏸️ PAUSE':
            self.titan.pause()
            self.btn_pause.description = '▶️ RESUME'
            self.btn_pause.button_style = 'success'
            self._log("⏸️ Paused")
        else:
            self.titan.resume()
            self.btn_pause.description = '⏸️ PAUSE'
            self.btn_pause.button_style = 'warning'
            self._log("▶️ Resumed")

    def _on_cancel(self, b):
        self.titan.cancel()
        self._log("❌ Cancelled by user")
        self.btn_start.disabled = False
        self.btn_pause.disabled = True
        self.btn_cancel.disabled = True

    def _on_analyze(self, b):
        url = self.url_input.value.strip().split('\n')[0].strip()
        if '|' in url:
            url = url.split('|')[0].strip()

        if not url:
            self._log("⚠️ Please enter a URL to analyze")
            return

        self._log(f"🔍 Analyzing: {url[:50]}...")

        # Detect protocol
        protocol = detect_protocol(url)
        self._log(f"📡 Protocol: {protocol.value}")

        if protocol == Protocol.M3U8:
            # Try to get qualities
            try:
                session = cffi_requests.Session(impersonate="chrome120")
                session.headers.update(DEFAULT_HEADERS)
                r = session.get(url, timeout=15, verify=False)

                if r.status_code == 200 and '#EXT-X-STREAM-INF' in r.text:
                    # Parse qualities
                    qualities = []
                    lines = r.text.splitlines()
                    base_url = url

                    for i, line in enumerate(lines):
                        if line.startswith('#EXT-X-STREAM-INF'):
                            bw_match = re.search(r'BANDWIDTH=(\d+)', line)
                            res_match = re.search(r'RESOLUTION=(\d+x\d+)', line)

                            bandwidth = int(bw_match.group(1)) if bw_match else 0
                            resolution = res_match.group(1) if res_match else "unknown"

                            if 'x' in resolution:
                                height = resolution.split('x')[1]
                                friendly_res = f"{height}p"
                            else:
                                friendly_res = resolution

                            if i + 1 < len(lines):
                                url_line = lines[i + 1].strip()
                                if url_line and not url_line.startswith('#'):
                                    q_url = urljoin(base_url, url_line)
                                    qualities.append(Quality(
                                        resolution=friendly_res,
                                        bandwidth=bandwidth,
                                        url=q_url
                                    ))

                    if qualities:
                        qualities = sorted(qualities, key=lambda q: q.bandwidth, reverse=True)
                        options = [('Auto (Best)', None)] + [(str(q), q) for q in qualities]
                        self.quality_dropdown.options = options
                        self._log(f"✅ Found {len(qualities)} quality options")
                    else:
                        self._log("ℹ️ No quality options (single stream)")
                else:
                    self._log("ℹ️ Single stream playlist")
            except Exception as e:
                self._log(f"⚠️ Analysis error: {e}")
        else:
            self._log(f"ℹ️ Direct download - no quality options")

    def _on_clear_log(self, b):
        self.log_output.clear_output()
        self._log("🗑️ Log cleared")

    def _on_show_history(self, b):
        history = self.titan.history.get_recent(10)
        self._log("=" * 40)
        self._log("📜 DOWNLOAD HISTORY (Recent 10)")
        self._log("=" * 40)

        for item in history:
            status_icon = "✅" if item['status'] == 'COMPLETED' else "❌"
            self._log(f"{status_icon} {item['filename']} | {item['status']} | {item['created_at']}")

        if not history:
            self._log("No download history yet")

    def _on_export_drive(self, b):
        try:
            from google.colab import drive
            drive.mount('/content/drive', force_remount=False)

            # Copy files to Drive
            drive_folder = '/content/drive/MyDrive/TitanDownloads'
            os.makedirs(drive_folder, exist_ok=True)

            count = 0
            for f in os.listdir(DOWNLOAD_DIR):
                src = os.path.join(DOWNLOAD_DIR, f)
                dst = os.path.join(drive_folder, f)
                if os.path.isfile(src):
                    shutil.copy2(src, dst)
                    count += 1

            self._log(f"💾 Exported {count} file(s) to Google Drive: /MyDrive/TitanDownloads/")
        except Exception as e:
            self._log(f"⚠️ Drive export error: {e}")

    def show(self):
        display(self.container)

# ==============================================================================
# 14. KEEP COLAB ALIVE + LAUNCH DASHBOARD
# ==============================================================================
dashboard = TitanDashboard()

# Session timer
session_start = datetime.now()

# Control widgets
keep_alive_toggle = widgets.ToggleButton(
    value=True,  # Default ON
    description='🔥 Keep Alive',
    button_style='success',
    tooltip='Keep Colab session alive',
    layout=widgets.Layout(width='130px')
)

auto_refresh_toggle = widgets.ToggleButton(
    value=True,  # Default ON
    description='⏱️ Auto-Refresh',
    button_style='info',
    layout=widgets.Layout(width='130px')
)

# Status display
status_output = widgets.HTML(value='<span style="color:#888;">⏳ Starting...</span>')
session_timer_display = widgets.HTML(value='<span style="color:#666;">Session: 00:00:00</span>')

# Counters
iteration_counter = [0]

def format_duration(seconds):
    """Format seconds to HH:MM:SS"""
    hours, remainder = divmod(int(seconds), 3600)
    minutes, secs = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

def keep_alive_loop():
    """Combined Keep-Alive and Status Update loop"""
    print(f"🔥 Keep-Alive started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("💡 Toggle 'Keep Alive' ON to prevent Colab from disconnecting")
    print("=" * 50)

    while True:
        try:
            current_time = datetime.now()
            iteration_counter[0] += 1

            # Update session timer
            elapsed = (current_time - session_start).total_seconds()
            session_timer_display.value = f'<span style="color:#00ff88;font-weight:bold;">🕐 Session: {format_duration(elapsed)}</span>'

            # Update status
            status_text = f'🔄 #{iteration_counter[0]} | {current_time.strftime("%H:%M:%S")}'
            if keep_alive_toggle.value:
                status_text += ' | 🔥 Alive'
            status_output.value = f'<span style="color:#00ff88;">{status_text}</span>'

            # Keep alive ping - print heartbeat every 5 minutes
            if keep_alive_toggle.value and iteration_counter[0] % 300 == 0:
                print(f"[{current_time.strftime('%Y-%m-%d %H:%M:%S')}] 🔥 Colab alive! Session: {format_duration(elapsed)} | Iterations: {iteration_counter[0]}")

            time.sleep(1)

        except Exception as e:
            status_output.value = f'<span style="color:#ff6b35;">⚠️ Error</span>'
            time.sleep(5)

# Start background thread
keep_alive_thread = threading.Thread(target=keep_alive_loop, daemon=True)
keep_alive_thread.start()

# Control panel
control_panel = widgets.HBox([
    keep_alive_toggle,
    auto_refresh_toggle,
    session_timer_display,
    status_output
], layout=widgets.Layout(
    margin='10px 0',
    padding='10px',
    gap='10px',
    border='1px solid rgba(0, 255, 136, 0.3)',
    border_radius='8px',
    background='rgba(0, 0, 0, 0.3)'
))

# Info message
info_msg = widgets.HTML('''
<div style="background:rgba(0,255,136,0.1);padding:10px;border-radius:8px;margin:10px 0;">
    <b style="color:#00ff88;">💡 Keep Colab Alive:</b><br>
    <span style="color:#aaa;font-size:12px;">
    • 🔥 <b>Keep Alive</b> = ป้องกัน Colab หลุด (เปิดไว้ตลอดขณะดาวน์โหลด)<br>
    • 🕐 <b>Session Timer</b> = แสดงเวลาที่รันมาแล้ว
    </span>
</div>
''')

# Display all
display(info_msg)
display(control_panel)
dashboard.show()

print(f"✅ Titan Downloader ready! Session started at {session_start.strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# @title 🎬 TITAN CONVERTER - MKV to MP4 (H.264) for Google Colab
# ==============================================================================
# High-Quality Video Converter with Original Quality Preservation
# Features: Batch Convert, Progress Tracking, Keep Colab Alive, Drive Export
# ==============================================================================

# 1. INSTALLATION
# ==============================================================================
!pip install -q tqdm ipywidgets

import os
import re
import time
import shutil
import subprocess
import threading
from datetime import datetime
from typing import Optional, List, Dict, Callable
from dataclasses import dataclass
from pathlib import Path

import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from tqdm.notebook import tqdm

# ==============================================================================
# 2. CONFIGURATION
# ==============================================================================
INPUT_DIR = "/content/input_videos"
OUTPUT_DIR = "/content/converted_videos"
TEMP_DIR = "/content/temp_convert"

os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(TEMP_DIR, exist_ok=True)

# ==============================================================================
# 3. FFMPEG PRESETS - Original Quality
# ==============================================================================
QUALITY_PRESETS = {
    "Original (CRF 18)": {
        "crf": "18",
        "preset": "slow",
        "description": "🎯 Visually lossless, best quality"
    },
    "High (CRF 20)": {
        "crf": "20",
        "preset": "medium",
        "description": "📺 Excellent quality, smaller size"
    },
    "Balanced (CRF 23)": {
        "crf": "23",
        "preset": "medium",
        "description": "⚖️ Good balance of quality/size"
    },
    "Fast (CRF 23)": {
        "crf": "23",
        "preset": "fast",
        "description": "⚡ Faster encoding, good quality"
    },
    "Copy (No Re-encode)": {
        "crf": None,
        "preset": None,
        "description": "📋 Just copy streams (fastest, if compatible)"
    }
}

# ==============================================================================
# 4. DATA CLASSES
# ==============================================================================
@dataclass
class VideoInfo:
    path: str
    filename: str
    size: int
    duration: float
    width: int
    height: int
    video_codec: str
    audio_codec: str
    bitrate: int

    @property
    def resolution(self) -> str:
        return f"{self.width}x{self.height}"

    @property
    def size_mb(self) -> float:
        return self.size / (1024 * 1024)

    @property
    def duration_str(self) -> str:
        mins, secs = divmod(int(self.duration), 60)
        hours, mins = divmod(mins, 60)
        if hours > 0:
            return f"{hours}:{mins:02d}:{secs:02d}"
        return f"{mins:02d}:{secs:02d}"

@dataclass
class ConvertJob:
    input_path: str
    output_path: str
    status: str = "pending"  # pending, converting, done, error
    progress: float = 0.0
    speed: str = ""
    eta: str = ""
    error: str = ""

# ==============================================================================
# 5. VIDEO ANALYZER
# ==============================================================================
def get_video_info(filepath: str) -> Optional[VideoInfo]:
    """Get video information using ffprobe"""
    try:
        cmd = [
            'ffprobe', '-v', 'quiet',
            '-print_format', 'json',
            '-show_format', '-show_streams',
            filepath
        ]
        result = subprocess.run(cmd, capture_output=True, text=True)

        import json
        data = json.loads(result.stdout)

        # Find video and audio streams
        video_stream = None
        audio_stream = None

        for stream in data.get('streams', []):
            if stream.get('codec_type') == 'video' and not video_stream:
                video_stream = stream
            elif stream.get('codec_type') == 'audio' and not audio_stream:
                audio_stream = stream

        if not video_stream:
            return None

        format_info = data.get('format', {})

        return VideoInfo(
            path=filepath,
            filename=os.path.basename(filepath),
            size=int(format_info.get('size', 0)),
            duration=float(format_info.get('duration', 0)),
            width=int(video_stream.get('width', 0)),
            height=int(video_stream.get('height', 0)),
            video_codec=video_stream.get('codec_name', 'unknown'),
            audio_codec=audio_stream.get('codec_name', 'unknown') if audio_stream else 'none',
            bitrate=int(format_info.get('bit_rate', 0))
        )
    except Exception as e:
        print(f"Error analyzing {filepath}: {e}")
        return None

def scan_input_folder() -> List[VideoInfo]:
    """Scan input folder for video files"""
    videos = []
    extensions = ['.mkv', '.avi', '.webm', '.mov', '.flv', '.wmv', '.m4v', '.ts']

    for filename in os.listdir(INPUT_DIR):
        if any(filename.lower().endswith(ext) for ext in extensions):
            filepath = os.path.join(INPUT_DIR, filename)
            info = get_video_info(filepath)
            if info:
                videos.append(info)

    return videos

# ==============================================================================
# 6. CONVERTER ENGINE
# ==============================================================================
class VideoConverter:
    def __init__(self):
        self.current_job: Optional[ConvertJob] = None
        self.log_callback: Callable = print
        self._cancelled = False

    def set_logger(self, callback: Callable):
        self.log_callback = callback

    def _log(self, message: str):
        self.log_callback(message)

    def cancel(self):
        self._cancelled = True

    def convert(self, input_path: str, output_path: str, preset_name: str,
                progress_callback: Callable = None) -> bool:
        """Convert video using FFmpeg"""
        self._cancelled = False
        preset = QUALITY_PRESETS.get(preset_name, QUALITY_PRESETS["Original (CRF 18)"])

        self.current_job = ConvertJob(input_path=input_path, output_path=output_path)
        self.current_job.status = "converting"

        # Get video duration for progress calculation
        info = get_video_info(input_path)
        total_duration = info.duration if info else 0

        # Build FFmpeg command
        if preset["crf"] is None:
            # Stream copy mode
            cmd = [
                'ffmpeg', '-y', '-i', input_path,
                '-c:v', 'copy', '-c:a', 'copy',
                '-movflags', '+faststart',
                output_path
            ]
            self._log(f"📋 Stream copy mode (no re-encoding)")
        else:
            # H.264 encoding with CRF
            cmd = [
                'ffmpeg', '-y', '-i', input_path,
                '-c:v', 'libx264',
                '-crf', preset["crf"],
                '-preset', preset["preset"],
                '-c:a', 'aac', '-b:a', '192k',
                '-movflags', '+faststart',
                '-progress', 'pipe:1',
                output_path
            ]
            self._log(f"🎬 Converting with CRF {preset['crf']}, preset: {preset['preset']}")

        try:
            process = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                universal_newlines=True
            )

            # Parse FFmpeg progress output
            while True:
                if self._cancelled:
                    process.kill()
                    self._log("❌ Conversion cancelled")
                    self.current_job.status = "error"
                    self.current_job.error = "Cancelled by user"
                    return False

                line = process.stdout.readline()
                if not line and process.poll() is not None:
                    break

                # Parse progress
                if 'out_time_ms=' in line:
                    match = re.search(r'out_time_ms=(\d+)', line)
                    if match and total_duration > 0:
                        current_ms = int(match.group(1))
                        current_sec = current_ms / 1000000
                        progress = min(current_sec / total_duration * 100, 100)
                        self.current_job.progress = progress
                        if progress_callback:
                            progress_callback(progress)

                elif 'speed=' in line:
                    match = re.search(r'speed=\s*([\d.]+)x', line)
                    if match:
                        self.current_job.speed = f"{match.group(1)}x"

            # Check result
            if process.returncode == 0:
                self.current_job.status = "done"
                self.current_job.progress = 100
                return True
            else:
                self.current_job.status = "error"
                self.current_job.error = f"FFmpeg error code: {process.returncode}"
                return False

        except Exception as e:
            self.current_job.status = "error"
            self.current_job.error = str(e)
            self._log(f"❌ Error: {e}")
            return False

# ==============================================================================
# 7. MODERN UI DASHBOARD
# ==============================================================================
class TitanConverterDashboard:
    def __init__(self):
        self.converter = VideoConverter()
        self.videos: List[VideoInfo] = []
        self.session_start = datetime.now()
        self._build_ui()
        self._setup_callbacks()
        self.converter.set_logger(self._log)

    def _build_ui(self):
        # CSS Styles
        self.styles = widgets.HTML('''
        <style>
            .converter-header {
                background: linear-gradient(90deg, #667eea, #764ba2);
                -webkit-background-clip: text;
                -webkit-text-fill-color: transparent;
                font-size: 28px;
                font-weight: 800;
                text-align: center;
                margin-bottom: 10px;
            }
            .converter-subtitle {
                color: #888;
                text-align: center;
                font-size: 12px;
                margin-bottom: 20px;
            }
            .video-card {
                background: rgba(102, 126, 234, 0.1);
                border: 1px solid rgba(102, 126, 234, 0.3);
                border-radius: 12px;
                padding: 12px;
                margin: 8px 0;
            }
            .video-name {
                font-size: 14px;
                font-weight: bold;
                color: #667eea;
            }
            .video-info {
                font-size: 12px;
                color: #aaa;
            }
            .stat-item {
                background: rgba(255,255,255,0.05);
                padding: 8px 12px;
                border-radius: 8px;
                margin: 4px;
            }
            .stat-value {
                font-size: 18px;
                font-weight: bold;
                color: #667eea;
            }
            .stat-label {
                font-size: 10px;
                color: #666;
            }
        </style>
        ''')

        # Header
        self.header = widgets.HTML('''
            <div class="converter-header">🎬 TITAN CONVERTER</div>
            <div class="converter-subtitle">MKV → MP4 (H.264) • Original Quality</div>
        ''')

        # Instructions
        self.instructions = widgets.HTML(f'''
        <div style="background:rgba(102,126,234,0.1);padding:15px;border-radius:12px;margin:10px 0;">
            <b style="color:#667eea;">📁 วิธีใช้:</b><br>
            <span style="color:#aaa;font-size:13px;">
            1. อัปโหลดไฟล์ MKV ไปที่ <code>{INPUT_DIR}</code><br>
            2. กด <b>🔍 SCAN</b> เพื่อค้นหาไฟล์<br>
            3. เลือก Preset คุณภาพ<br>
            4. กด <b>🚀 CONVERT ALL</b>
            </span>
        </div>
        ''')

        # Quality preset dropdown
        self.preset_label = widgets.HTML('<b style="color:#888;">Quality Preset:</b>')
        self.preset_dropdown = widgets.Dropdown(
            options=[(f"{name} - {p['description']}", name) for name, p in QUALITY_PRESETS.items()],
            value="Original (CRF 18)",
            layout=widgets.Layout(width='400px')
        )

        # File upload
        self.file_upload = widgets.FileUpload(
            accept='.mkv,.avi,.webm,.mov,.flv,.wmv,.m4v,.ts',
            multiple=True,
            description='📁 Upload Videos'
        )

        # Stats
        self.stat_total = widgets.HTML('<div class="stat-item"><div class="stat-value">0</div><div class="stat-label">Total Files</div></div>')
        self.stat_size = widgets.HTML('<div class="stat-item"><div class="stat-value">0 MB</div><div class="stat-label">Total Size</div></div>')
        self.stat_converted = widgets.HTML('<div class="stat-item"><div class="stat-value">0</div><div class="stat-label">Converted</div></div>')
        self.stat_session = widgets.HTML('<div class="stat-item"><div class="stat-value">00:00</div><div class="stat-label">Session</div></div>')

        self.stats_row = widgets.HBox([
            self.stat_total, self.stat_size, self.stat_converted, self.stat_session
        ], layout=widgets.Layout(justify_content='space-around', margin='10px 0'))

        # Control buttons
        self.btn_scan = widgets.Button(
            description='🔍 SCAN',
            button_style='info',
            layout=widgets.Layout(width='100px', height='40px')
        )
        self.btn_upload = widgets.Button(
            description='📁 UPLOAD',
            button_style='primary',
            layout=widgets.Layout(width='100px', height='40px')
        )
        self.btn_convert = widgets.Button(
            description='🚀 CONVERT ALL',
            button_style='success',
            layout=widgets.Layout(width='140px', height='40px')
        )
        self.btn_cancel = widgets.Button(
            description='❌ CANCEL',
            button_style='danger',
            layout=widgets.Layout(width='100px', height='40px'),
            disabled=True
        )

        self.controls = widgets.HBox([
            self.btn_scan, self.btn_upload, self.btn_convert, self.btn_cancel
        ], layout=widgets.Layout(justify_content='center', gap='10px', margin='15px 0'))

        # Progress bar
        self.progress_bar = widgets.FloatProgress(
            value=0, min=0, max=100,
            bar_style='info',
            layout=widgets.Layout(width='100%', height='25px')
        )
        self.progress_label = widgets.HTML('<div style="text-align:center;color:#888;font-size:12px;">Ready</div>')

        # Video list
        self.video_list = widgets.VBox([], layout=widgets.Layout(
            width='100%',
            max_height='250px',
            overflow_y='auto'
        ))

        # Log output
        self.log_output = widgets.Output(layout=widgets.Layout(
            height='150px',
            overflow_y='auto',
            border='1px solid rgba(102, 126, 234, 0.2)',
            border_radius='8px',
            padding='10px',
            background_color='#0a0a0f'
        ))

        # Footer buttons
        self.btn_export_drive = widgets.Button(
            description='💾 Export to Drive',
            layout=widgets.Layout(width='140px')
        )
        self.btn_clear_log = widgets.Button(
            description='🗑️ Clear Log',
            layout=widgets.Layout(width='100px')
        )

        self.footer = widgets.HBox([
            self.btn_export_drive, self.btn_clear_log
        ], layout=widgets.Layout(justify_content='flex-end', gap='10px', margin='10px 0'))

        # Assemble UI
        self.container = widgets.VBox([
            self.styles,
            self.header,
            self.instructions,
            widgets.HBox([self.preset_label, self.preset_dropdown],
                        layout=widgets.Layout(align_items='center', gap='10px', margin='10px 0')),
            self.file_upload,
            self.stats_row,
            self.controls,
            self.progress_bar,
            self.progress_label,
            widgets.HTML('<div style="color:#667eea;font-size:14px;margin:10px 0;">📋 Video Files</div>'),
            self.video_list,
            widgets.HTML('<div style="color:#667eea;font-size:14px;margin:10px 0;">📋 Logs</div>'),
            self.log_output,
            self.footer
        ], layout=widgets.Layout(
            padding='20px',
            border='2px solid rgba(102, 126, 234, 0.3)',
            border_radius='16px',
            width='850px',
            background='linear-gradient(135deg, #0a0a0f 0%, #1a1a2e 100%)'
        ))

    def _setup_callbacks(self):
        self.btn_scan.on_click(self._on_scan)
        self.btn_upload.on_click(self._on_upload)
        self.btn_convert.on_click(self._on_convert)
        self.btn_cancel.on_click(self._on_cancel)
        self.btn_clear_log.on_click(self._on_clear_log)
        self.btn_export_drive.on_click(self._on_export_drive)

    def _log(self, message: str):
        with self.log_output:
            timestamp = datetime.now().strftime("%H:%M:%S")
            print(f"[{timestamp}] {message}")

    def _update_stats(self):
        total = len(self.videos)
        total_size = sum(v.size_mb for v in self.videos)

        self.stat_total.value = f'<div class="stat-item"><div class="stat-value">{total}</div><div class="stat-label">Total Files</div></div>'
        self.stat_size.value = f'<div class="stat-item"><div class="stat-value">{total_size:.1f} MB</div><div class="stat-label">Total Size</div></div>'

        # Update session timer
        elapsed = (datetime.now() - self.session_start).total_seconds()
        mins, secs = divmod(int(elapsed), 60)
        hours, mins = divmod(mins, 60)
        time_str = f"{hours}:{mins:02d}:{secs:02d}" if hours > 0 else f"{mins:02d}:{secs:02d}"
        self.stat_session.value = f'<div class="stat-item"><div class="stat-value">{time_str}</div><div class="stat-label">Session</div></div>'

    def _update_video_list(self):
        cards = []
        for video in self.videos:
            card_html = f'''
            <div class="video-card">
                <div class="video-name">🎬 {video.filename}</div>
                <div class="video-info">
                    📐 {video.resolution} |
                    ⏱️ {video.duration_str} |
                    📦 {video.size_mb:.1f} MB |
                    🎥 {video.video_codec} |
                    🔊 {video.audio_codec}
                </div>
            </div>
            '''
            cards.append(widgets.HTML(card_html))

        if cards:
            self.video_list.children = tuple(cards)
        else:
            self.video_list.children = (widgets.HTML(f'<div style="color:#666;text-align:center;padding:20px;">No videos found. Upload files or place them in {INPUT_DIR}</div>'),)

    def _on_scan(self, b):
        self._log("🔍 Scanning for video files...")
        self.videos = scan_input_folder()
        self._update_stats()
        self._update_video_list()
        self._log(f"✅ Found {len(self.videos)} video file(s)")

    def _on_upload(self, b):
        if not self.file_upload.value:
            self._log("⚠️ Please select files first")
            return

        count = 0
        for file_info in self.file_upload.value:
            try:
                filename = file_info['name'] if isinstance(file_info, dict) else file_info.name
                content = file_info['content'] if isinstance(file_info, dict) else file_info.content

                filepath = os.path.join(INPUT_DIR, filename)
                with open(filepath, 'wb') as f:
                    f.write(content)
                count += 1
                self._log(f"📁 Uploaded: {filename}")
            except Exception as e:
                self._log(f"❌ Upload error: {e}")

        if count > 0:
            self._log(f"✅ Uploaded {count} file(s)")
            self._on_scan(None)

        self.file_upload.value = ()

    def _on_convert(self, b):
        if not self.videos:
            self._log("⚠️ No videos to convert. Click SCAN first.")
            return

        self.btn_convert.disabled = True
        self.btn_cancel.disabled = False

        preset_name = self.preset_dropdown.value
        self._log(f"🚀 Starting conversion with preset: {preset_name}")

        converted = 0
        total = len(self.videos)

        for i, video in enumerate(self.videos):
            if self.converter._cancelled:
                break

            # Update progress label
            self.progress_label.value = f'<div style="text-align:center;color:#667eea;font-size:12px;">Converting {i+1}/{total}: {video.filename}</div>'

            # Output path
            output_name = Path(video.filename).stem + ".mp4"
            output_path = os.path.join(OUTPUT_DIR, output_name)

            self._log(f"▶️ [{i+1}/{total}] {video.filename}")

            def update_progress(progress):
                self.progress_bar.value = progress

            success = self.converter.convert(
                video.path, output_path, preset_name, update_progress
            )

            if success:
                converted += 1
                # Get output size
                if os.path.exists(output_path):
                    out_size = os.path.getsize(output_path) / (1024*1024)
                    self._log(f"✅ Done: {output_name} ({out_size:.1f} MB)")
            else:
                self._log(f"❌ Failed: {video.filename}")

            self.stat_converted.value = f'<div class="stat-item"><div class="stat-value">{converted}/{total}</div><div class="stat-label">Converted</div></div>'

        self.btn_convert.disabled = False
        self.btn_cancel.disabled = True
        self.progress_label.value = f'<div style="text-align:center;color:#00ff88;font-size:12px;">✅ Completed! {converted}/{total} converted</div>'
        self._log(f"🏁 Conversion complete! {converted}/{total} files converted")
        self._log(f"📁 Output folder: {OUTPUT_DIR}")

    def _on_cancel(self, b):
        self.converter.cancel()
        self._log("❌ Cancelling...")

    def _on_clear_log(self, b):
        self.log_output.clear_output()
        self._log("🗑️ Log cleared")

    def _on_export_drive(self, b):
        try:
            from google.colab import drive
            drive.mount('/content/drive', force_remount=False)

            drive_folder = '/content/drive/MyDrive/TitanConverter'
            os.makedirs(drive_folder, exist_ok=True)

            count = 0
            for f in os.listdir(OUTPUT_DIR):
                src = os.path.join(OUTPUT_DIR, f)
                dst = os.path.join(drive_folder, f)
                if os.path.isfile(src):
                    shutil.copy2(src, dst)
                    count += 1

            self._log(f"💾 Exported {count} file(s) to Google Drive: /MyDrive/TitanConverter/")
        except Exception as e:
            self._log(f"⚠️ Drive export error: {e}")

    def show(self):
        display(self.container)
        self._log("🎬 Titan Converter พร้อมใช้งาน!")
        self._log(f"📁 Input folder: {INPUT_DIR}")
        self._log(f"📁 Output folder: {OUTPUT_DIR}")

# ==============================================================================
# 8. KEEP COLAB ALIVE + LAUNCH
# ==============================================================================
dashboard = TitanConverterDashboard()

# Session timer
session_start = datetime.now()
iteration_counter = [0]

# Keep alive widgets
keep_alive_toggle = widgets.ToggleButton(
    value=True,
    description='🔥 Keep Alive',
    button_style='success',
    layout=widgets.Layout(width='130px')
)

status_display = widgets.HTML(value='<span style="color:#888;">⏳ Starting...</span>')

def format_duration(seconds):
    hours, remainder = divmod(int(seconds), 3600)
    minutes, secs = divmod(remainder, 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

def keep_alive_loop():
    print(f"🔥 Keep-Alive started at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 50)

    while True:
        try:
            iteration_counter[0] += 1
            current_time = datetime.now()
            elapsed = (current_time - session_start).total_seconds()

            if keep_alive_toggle.value:
                status_display.value = f'<span style="color:#00ff88;">🕐 {format_duration(elapsed)} | 🔥 Alive #{iteration_counter[0]}</span>'

                if iteration_counter[0] % 300 == 0:
                    print(f"[{current_time.strftime('%Y-%m-%d %H:%M:%S')}] 🔥 Colab alive! Session: {format_duration(elapsed)}")

            # Update dashboard stats
            dashboard._update_stats()

            time.sleep(1)
        except:
            time.sleep(5)

# Start keep-alive thread
keep_alive_thread = threading.Thread(target=keep_alive_loop, daemon=True)
keep_alive_thread.start()

# Control panel
control_panel = widgets.HBox([
    keep_alive_toggle,
    status_display
], layout=widgets.Layout(
    margin='10px 0',
    padding='10px',
    gap='15px',
    border='1px solid rgba(102, 126, 234, 0.3)',
    border_radius='8px',
    background='rgba(0, 0, 0, 0.3)'
))

# Display
display(control_panel)
dashboard.show()

print(f"✅ Titan Converter ready! Session started at {session_start.strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# @title 🚀 Elite Douyin/TikTok Live Downloader (v1.1 - Deep Scan Fixed)
import os
import sys
import json
import time
import shutil
import subprocess
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import google.colab.files

# --- 🎨 CSS & AESTHETICS ---
def inject_css():
    style = """
    <style>
        :root {
            --glass-bg: rgba(16, 18, 27, 0.95);
            --glass-border: rgba(255, 255, 255, 0.1);
            --neon-blue: #00f2ff;
            --neon-purple: #bc13fe;
            --text-main: #ffffff;
            --text-sub: #a0a0a0;
        }
        .elite-container {
            background: var(--glass-bg);
            border: 1px solid var(--glass-border);
            border-radius: 16px;
            padding: 20px;
            box-shadow: 0 8px 32px 0 rgba(0, 0, 0, 0.37);
            margin-bottom: 20px;
        }
        .jupyter-button {
            background: linear-gradient(135deg, var(--neon-blue) 0%, #00a8ff 100%) !important;
            border: none !important; border-radius: 8px !important; color: #000 !important;
            font-weight: 700 !important; transition: all 0.3s ease !important;
        }
        .jupyter-button:hover { transform: translateY(-2px); box-shadow: 0 0 15px var(--neon-blue); }
        .jupyter-button.mod-danger { background: linear-gradient(135deg, #ff4757 0%, #ff6b81 100%) !important; color: white !important; }
        .jupyter-button.mod-success { background: linear-gradient(135deg, #2ecc71 0%, #26de81 100%) !important; }
        input[type="text"] {
            background: rgba(255, 255, 255, 0.05) !important;
            border: 1px solid var(--glass-border) !important;
            color: var(--text-main) !important;
            border-radius: 6px !important; padding: 8px !important;
        }
        h2.elite-title {
            background: -webkit-linear-gradient(var(--neon-blue), var(--neon-purple));
            -webkit-background-clip: text; -webkit-text-fill-color: transparent;
            font-family: 'Roboto', sans-serif; font-weight: 900; margin-bottom: 10px;
        }
        /* Scrollbars */
        ::-webkit-scrollbar { width: 8px; }
        ::-webkit-scrollbar-track { background: transparent; }
        ::-webkit-scrollbar-thumb { background: var(--glass-border); border-radius: 4px; }
    </style>
    """
    display(HTML(style))

# --- 🔧 CONFIG & UTILS ---
class Config:
    DEFAULT_DIR = "/content/DouyinLiveRecorder/downloads/抖音直播/"
    BATCH_LIMIT = 3
    MIN_DURATION_SEC = 600 # 10 Minutes
    EXTENSIONS = ('.mp4', '.avi', '.mkv', '.mov', '.wmv', '.flv', '.webm', '.ts')
    HISTORY_FILE = "download_history.json"

    @staticmethod
    def save_history(file_path):
        history = Config.load_history()
        history.add(file_path)
        with open(Config.HISTORY_FILE, 'w') as f:
            json.dump(list(history), f)

    @staticmethod
    def load_history():
        if os.path.exists(Config.HISTORY_FILE):
            try:
                with open(Config.HISTORY_FILE, 'r') as f:
                    return set(json.load(f))
            except: return set()
        return set()

class SystemUtils:
    @staticmethod
    def install_ffmpeg():
        if not shutil.which('ffprobe'):
            print("⚙️ Installing FFmpeg...")
            subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], stdout=subprocess.DEVNULL)
            print("✅ FFmpeg Installed.")

    @staticmethod
    def get_duration(file_path):
        duration = 0.0
        ffprobe_cmd = shutil.which('ffprobe')
        if ffprobe_cmd:
            try:
                cmd = [ffprobe_cmd, '-v', 'error', '-show_entries', 'format=duration', '-of', 'json', file_path]
                result = subprocess.run(cmd, capture_output=True, text=True, timeout=10)
                data = json.loads(result.stdout)
                duration = float(data['format']['duration'])
            except: duration = SystemUtils._simulate_duration(file_path)
        else: duration = SystemUtils._simulate_duration(file_path)
        return duration, SystemUtils.format_time(duration)

    @staticmethod
    def _simulate_duration(file_path):
        try: return os.path.getsize(file_path) / (1024 * 1024 * 0.2)
        except: return 0.0

    @staticmethod
    def format_time(seconds):
        m, s = divmod(seconds, 60); h, m = divmod(m, 60)
        return f"{int(h):02d}h {int(m):02d}m {int(s):02d}s"

    @staticmethod
    def format_size(size):
        for unit in ['B', 'KB', 'MB', 'GB']:
            if size < 1024: return f"{size:.2f} {unit}"
            size /= 1024
        return f"{size:.2f} TB"

# --- 🖥️ APP CLASS ---
class EliteDownloaderApp:
    def __init__(self):
        inject_css()
        SystemUtils.install_ffmpeg()
        self.download_dir = Config.DEFAULT_DIR
        self.files_data = []; self.checkbox_map = {}; self.history = Config.load_history()

        # UI Construction
        self.log_output = widgets.Output(layout=widgets.Layout(height='150px', overflow_y='scroll', border='1px solid #333', padding='10px'))
        self.file_list_box = widgets.VBox([])

        # Header
        self.header = widgets.VBox([
            widgets.HTML('<h2 class="elite-title">🚀 Elite Downloader v1.1</h2>'),
            widgets.HTML(f"<span style='color:#777'>System: {'✅ FFprobe Ready' if shutil.which('ffprobe') else '⚠️ Simulating Duration'}</span>")
        ])

        # Controls
        self.path_input = widgets.Text(value=self.download_dir, description='📂 Path:', placeholder='/content/...')
        self.path_input.observe(lambda c: setattr(self, 'download_dir', c['new']), names='value')

        self.btn_scan = widgets.Button(description="🔄 Deep Scan", icon='search', layout=widgets.Layout(width='auto'))
        self.btn_scan.add_class('jupyter-button')
        self.btn_scan.on_click(lambda b: self._refresh_file_list())

        self.btn_dl = widgets.Button(description="⬇️ Download Batch", icon='download', layout=widgets.Layout(width='auto'))
        self.btn_dl.add_class('jupyter-button'); self.btn_dl.add_class('mod-success')
        self.btn_dl.on_click(self._on_batch_download)

        self.btn_reset = widgets.Button(description="🗑️ Reset History", icon='trash')
        self.btn_reset.add_class('jupyter-button'); self.btn_reset.add_class('mod-danger')
        self.btn_reset.on_click(self._clear_history)

        self.controls = widgets.VBox([
            widgets.HTML("<div class='elite-container'>"),
            self.path_input,
            widgets.HBox([self.btn_scan, self.btn_dl, self.btn_reset], layout=widgets.Layout(margin='10px 0')),
            widgets.HTML("</div>")
        ])

        self.main_layout = widgets.VBox([self.header, self.controls, widgets.HTML("<h3>📄 Video List</h3>"), self.file_list_box, widgets.HTML("<hr>"), self.log_output])

        # Auto-start scan
        self._refresh_file_list()

    def _log(self, msg, level="INFO"):
        color = {"INFO": "white", "SUCCESS": "#00ff9d", "ERROR": "#ff4757", "WARN": "orange"}.get(level, "white")
        with self.log_output: print(f"[{time.strftime('%H:%M:%S')}] {msg}")

    def _refresh_file_list(self):
        self._log(f"🕵️ Deep scanning: {self.download_dir}...", "INFO")
        self.files_data = []
        if not os.path.exists(self.download_dir):
            self._log(f"Path not found: {self.download_dir}", "ERROR"); return

        found_files = []
        # RECURSIVE SEARCH FIX
        for root, _, files in os.walk(self.download_dir):
            for file in files:
                if file.lower().endswith(Config.EXTENSIONS):
                    found_files.append(os.path.join(root, file))

        if not found_files:
            self._log("No video files found in tree.", "WARN");
            self.file_list_box.children = [widgets.HTML("📂 Folder tree is empty.")]; return

        found_files.sort()
        self._log(f"Found {len(found_files)} potential files. Analyzing...", "INFO")

        items = []
        self.checkbox_map = {}

        with self.log_output: # Capture stdout from any subprocess noise
            for full_path in found_files:
                d, fd = SystemUtils.get_duration(full_path)
                if d < Config.MIN_DURATION_SEC: continue

                f_info = {
                    "name": os.path.basename(full_path),
                    "path": full_path,
                    "rel_path": full_path.replace(self.download_dir, '...'),
                    "size": SystemUtils.format_size(os.path.getsize(full_path)),
                    "duration": fd
                }

                # Create UI Element
                is_done = f_info['path'] in self.history
                chk = widgets.Checkbox(value=not is_done, layout=widgets.Layout(width='30px'), indent=False)
                self.checkbox_map[chk] = f_info

                btn = widgets.Button(description="⬇️", layout=widgets.Layout(width='40px'))
                btn.on_click(lambda b, f=f_info: self._dl_single(f))

                row = widgets.HBox([
                    chk,
                    widgets.HTML(f"<div style='margin-left:10px'><b style='color:#00f2ff'>{f_info['name']}</b><br><span style='font-size:0.8em;color:#888'>{f_info['rel_path']} | {f_info['size']} | {f_info['duration']}</span></div>"),
                    btn
                ], layout=widgets.Layout(border_bottom='1px solid #333', padding='4px'))
                items.append(row)

        self.file_list_box.children = items if items else [widgets.HTML("⚠️ All files are shorter than 10 mins.")]
        if items: self._log(f"✅ Listed {len(items)} valid videos.", "SUCCESS")

    def _dl_single(self, f):
        self._log(f"Starting: {f['name']}", "INFO")
        try: google.colab.files.download(f['path']); self.history.add(f['path']); Config.save_history(f['path'])
        except Exception as e: self._log(f"Error: {e}", "ERROR")

    def _on_batch_download(self, b):
        b.disabled = True
        selected = [self.checkbox_map[c] for c in self.checkbox_map if c.value and self.checkbox_map[c]['path'] not in self.history]
        self._log(f"Batch processing {min(len(selected), Config.BATCH_LIMIT)} files...", "INFO")

        for i, f in enumerate(selected[:Config.BATCH_LIMIT]):
            self._log(f"({i+1}) Downloading: {f['name']}", "INFO")
            try:
                google.colab.files.download(f['path'])
                self.history.add(f['path']); Config.save_history(f['path'])
                time.sleep(2)
            except Exception as e: self._log(f"Fail: {e}", "ERROR")
        b.disabled = False

    def _clear_history(self, b):
        if os.path.exists(Config.HISTORY_FILE): os.remove(Config.HISTORY_FILE)
        self.history = set(); self._refresh_file_list(); self._log("History cleared.", "WARN")

# --- 🚀 RUN ---
app = EliteDownloaderApp()
display(app.main_layout)


In [ ]:
# @title 🛡️ Ultimate Elite Framework v2.5 (Anti-Ghost Protection)
# @markdown **Run this cell** to start.
# @markdown <br>🔰 **Upgrade:** Added "Anti-Ghost" logic to prevent local file traps.
# @markdown <br>⚡ **Rule:** Videos shorter than 10 minutes are hidden.

import os
import sys
import shutil
import subprocess
import logging
import hashlib
from datetime import timedelta
from dataclasses import dataclass, field
from typing import List, Optional, Tuple
from pathlib import Path

# --- 1. Dependency Check & Installation ---
def install_dependencies():
    """Installs required third-party libraries."""
    try:
        import rich
    except ImportError:
        print("✨ Installing UI dependencies...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "rich"])

install_dependencies()

from google.colab import drive
from rich.console import Console
from rich.table import Table
from rich.progress import Progress, SpinnerColumn, BarColumn, TextColumn, TimeRemainingColumn, FileSizeColumn
from rich.panel import Panel
from rich.logging import RichHandler
from rich.prompt import Prompt, Confirm
from rich import box
from rich.theme import Theme
from rich.style import Style

# --- 2. Configuration & Theming (NEON MODE) ---

custom_theme = Theme({
    "info": "bold cyan",
    "warning": "bold yellow",
    "danger": "bold bright_red",
    "success": "bold spring_green1",
    "header": "bold white",
    "key": "bold turquoise2",
    "value": "bold orchid1",
    "panel.border": "spring_green3",
})

@dataclass
class AppConfig:
    """Centralized Configuration."""
    source_dir: Path = Path('/content/DouyinLiveRecorder/downloads/抖音直播')
    target_dir: Path = Path('/content/drive/MyDrive/8888/ColabDL')
    min_duration_sec: int = 600
    enable_checksum: bool = True
    log_file: str = 'backup_log.log'
    default_op: str = 'move'

@dataclass
class VideoFile:
    index: int
    path: Path
    name: str
    size_mb: float
    duration_sec: float
    duration_str: str

# --- 3. Utilities ---

class LoggerSetup:
    @staticmethod
    def setup(log_path: Path, console: Console):
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)

        logging.basicConfig(
            level="INFO",
            format="%(message)s",
            datefmt="[%X]",
            handlers=[
                RichHandler(console=console, rich_tracebacks=True, show_path=False, markup=True),
                logging.FileHandler(log_path)
            ]
        )
        return logging.getLogger("rich")

class SystemUtils:
    @staticmethod
    def install_ffmpeg(logger):
        try:
            subprocess.run(['ffprobe', '-h'], check=True, capture_output=True)
        except (subprocess.CalledProcessError, FileNotFoundError):
            logger.warning("[warning]FFmpeg not found. Installing...[/]")
            try:
                subprocess.run(['apt-get', 'update'], check=True, capture_output=True)
                subprocess.run(['apt-get', 'install', '-y', 'ffmpeg'], check=True, capture_output=True)
                logger.info("[success]FFmpeg installed successfully.[/]")
            except Exception as e:
                logger.error(f"[danger]Failed to install FFmpeg: {e}[/]")

    @staticmethod
    def calculate_md5(file_path: Path, chunk_size=8192) -> Optional[str]:
        try:
            hasher = hashlib.md5()
            with open(file_path, 'rb') as f:
                for chunk in iter(lambda: f.read(chunk_size), b''):
                    hasher.update(chunk)
            return hasher.hexdigest()
        except Exception:
            return None

    @staticmethod
    def get_duration(file_path: Path) -> float:
        try:
            cmd = ['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', str(file_path)]
            result = subprocess.run(cmd, check=True, capture_output=True, text=True)
            val = result.stdout.strip()
            return float(val) if val else 0.0
        except Exception:
            return 0.0

    @staticmethod
    def format_seconds(seconds: float) -> str:
        return str(timedelta(seconds=int(seconds)))

# --- 4. Main Application ---

class BackupApp:
    def __init__(self):
        self.console = Console(theme=custom_theme)
        self.config = AppConfig()

        # Initial Logging setup (Temporary until drive mounts)
        self.logger = LoggerSetup.setup(Path('/content/backup_log.log'), self.console)
        self.video_files: List[VideoFile] = []

    def mount_drive(self):
        """
        🛡️ ANTI-GHOST MOUNTING PROTOCOL
        Ensures a clean, real connection to Google Drive.
        """
        mount_point = Path('/content/drive')

        # 1. Check for "Ghost Folder" (Exists but not a mount point)
        if mount_point.exists() and not os.path.ismount(mount_point):
            self.console.print("[warning]⚠️ Detect Ghost Folder! Cleaning up to allow real connection...[/]")
            try:
                # Remove the fake folder blocking the mount
                shutil.rmtree(mount_point)
                mount_point.mkdir()
            except Exception as e:
                self.logger.error(f"Failed to clear ghost folder: {e}")

        # 2. Perform Mount
        if not os.path.ismount(mount_point):
            self.console.print(Panel("🌥️ Connecting to Google Drive...", style="panel.border"))
            try:
                drive.mount('/content/drive', force_remount=True)
            except Exception as e:
                self.console.print(f"[danger]❌ Mount Failed: {e}[/]")
                return

        # 3. Verify Connection
        if os.path.ismount(mount_point):
            self.logger.info("[info]✅ Google Drive connected securely.[/]")
            # Re-init logger to save to Drive now that it's safe
            self.config.target_dir.mkdir(parents=True, exist_ok=True)
            self.logger = LoggerSetup.setup(self.config.target_dir / self.config.log_file, self.console)
        else:
            self.console.print("[danger]🚨 CRITICAL: Drive is NOT mounted. Operations aborted.[/]")

    def scan_files(self):
        if not self.config.source_dir.exists():
            self.console.print(Panel(
                f"[danger]❌ Source Directory Not Found[/]\nPath: [bold white]{self.config.source_dir}[/]\n\n"
                "[white]Please verify your download path.[/]",
                title="Configuration Error", style="danger"
            ))
            return

        all_files = []
        stats = {'found': 0, 'filtered_out': 0}

        with Progress(
            SpinnerColumn(style="bold spring_green1"),
            TextColumn("[bold cyan]{task.description}"),
            transient=True,
            console=self.console
        ) as progress:
            progress.add_task(f"Scanning (Ignoring < {self.config.min_duration_sec/60} mins)...", total=None)

            idx_counter = 0
            for file_path in self.config.source_dir.rglob('*'):
                if file_path.is_file():
                    if file_path.suffix.lower() not in ['.mp4', '.ts']:
                        continue

                    dur = SystemUtils.get_duration(file_path)
                    if dur < self.config.min_duration_sec:
                        stats['filtered_out'] += 1
                        continue

                    size_mb = file_path.stat().st_size / (1024 * 1024)
                    all_files.append(VideoFile(idx_counter, file_path, file_path.name, size_mb, dur, SystemUtils.format_seconds(dur)))
                    idx_counter += 1

        self.video_files = all_files

        if not all_files:
            msg = f"No videos longer than {self.config.min_duration_sec/60} minutes found."
            if stats['filtered_out'] > 0:
                msg += f"\n(Hidden {stats['filtered_out']} short files)"

            self.console.print(Panel(
                f"📂 Path: [bold white]{self.config.source_dir}[/]\n{msg}",
                title="⚠️ No Eligible Files", style="warning"
            ))
        else:
            self.logger.info(f"[success]Found {len(self.video_files)} eligible video files (> 10 mins).[/]")

    def display_files(self):
        if not self.video_files: return False

        table = Table(
            title=f"🎬 Eligible Videos (>10 mins) ({len(self.video_files)})",
            box=box.ROUNDED,
            header_style="header",
            border_style="spring_green3",
            show_lines=True
        )
        table.add_column("ID", style="key", justify="center")
        table.add_column("File Name", style="bold white")
        table.add_column("Size (MB)", style="success", justify="right")
        table.add_column("Duration", style="value", justify="center")

        for vid in self.video_files:
            table.add_row(str(vid.index), vid.name, f"{vid.size_mb:.2f}", vid.duration_str)

        self.console.print(table)
        return True

    def process_files(self):
        if not self.display_files(): return

        # 🔒 FINAL SAFETY CHECK BEFORE ACTION
        if not os.path.ismount('/content/drive'):
            self.console.print("[danger]🚨 Drive disconnected! Attempting to reconnect...[/]")
            self.mount_drive()
            if not os.path.ismount('/content/drive'):
                self.console.print("[danger]❌ Aborting: Cannot establish connection to Drive.[/]")
                return

        self.console.print("\n[bold white]⚙️ Action Required[/]")
        op_mode = Prompt.ask("Select Operation", choices=["copy", "move"], default=self.config.default_op, console=self.console)
        selection = Prompt.ask("Select Index ([spring_green1]all[/] or 1,2,3)", console=self.console)

        selected_videos = self.video_files if selection.lower() == 'all' else []
        if selection.lower() != 'all':
            try:
                indices = [int(x.strip()) for x in selection.split(',')]
                selected_videos = [v for v in self.video_files if v.index in indices]
            except:
                self.console.print("[danger]Invalid input![/]")
                return

        if not selected_videos: return

        stats = {'success': 0, 'fail': 0, 'skip': 0, 'mismatch': 0}

        with Progress(
            SpinnerColumn(style="bold spring_green1"),
            BarColumn(bar_width=None, style="grey23", complete_style="bold spring_green1"),
            TextColumn("[bold white]{task.description}"),
            FileSizeColumn(),
            console=self.console
        ) as progress:

            task_id = progress.add_task(f"[bold]{op_mode.capitalize()}ing...[/]", total=len(selected_videos))

            for vid in selected_videos:
                dst_path = self.config.target_dir / vid.name

                if dst_path.exists():
                    self.logger.info(f"[grey50]⏭️ Skipped {vid.name} (Exists)[/]")
                    stats['skip'] += 1
                    progress.advance(task_id)
                    continue

                try:
                    src_hash = SystemUtils.calculate_md5(vid.path) if self.config.enable_checksum else None

                    progress.update(task_id, description=f"{op_mode.capitalize()}: [cyan]{vid.name}[/]")
                    if op_mode == 'copy': shutil.copy2(vid.path, dst_path)
                    else: shutil.move(str(vid.path), str(dst_path))

                    if self.config.enable_checksum:
                        progress.update(task_id, description=f"Verifying: [cyan]{vid.name}[/]")
                        if src_hash == SystemUtils.calculate_md5(dst_path):
                            self.logger.info(f"[success]✔ Verified: {vid.name}[/]")
                            stats['success'] += 1
                        else:
                            self.logger.error(f"[danger]❌ Mismatch: {vid.name}[/]")
                            stats['mismatch'] += 1
                    else:
                        stats['success'] += 1

                except Exception as e:
                    self.logger.error(f"[danger]Failed {vid.name}: {e}[/]")
                    stats['fail'] += 1

                progress.advance(task_id)

        if op_mode == 'copy' and stats['success'] > 0:
            if Confirm.ask("🗑️ Delete source files?", console=self.console):
                self.delete_sources(selected_videos, self.config.target_dir)

        self.print_summary(stats, op_mode)

    def delete_sources(self, videos, target_dir):
        deleted = 0
        for vid in videos:
            if (target_dir / vid.name).exists():
                os.remove(vid.path)
                deleted += 1
        self.console.print(f"[success]🗑️ Cleaned up {deleted} files.[/]")

    def print_summary(self, stats, op_mode):
        grid = Table.grid(expand=True, padding=(0, 2))
        grid.add_column()
        grid.add_column(justify="right")

        grid.add_row(f"[bold cyan]Successfully {op_mode}ed[/]", f"[bold spring_green1]{stats['success']}[/]")
        if stats['fail']: grid.add_row("[bold red]Failed[/]", f"[bold red]{stats['fail']}[/]")
        if stats['skip']: grid.add_row("[grey50]Skipped[/]", f"[grey50]{stats['skip']}[/]")

        self.console.print(Panel(grid, title="📊 Summary", border_style="spring_green3", expand=False))
        self.console.print("[bold spring_green1]✨ All tasks completed successfully![/]")

    def run(self):
        self.console.print(Panel.fit("🛡️ [bold spring_green1]Google Drive Backup Manager[/] [bold white]Anti-Ghost Edition[/]", border_style="spring_green3"))
        SystemUtils.install_ffmpeg(self.logger)
        self.mount_drive()
        self.scan_files()
        self.process_files()
        try: drive.flush_and_unmount(); self.console.print("[grey50 italic]Drive unmounted.[/]")
        except: pass

if __name__ == "__main__":
    app = BackupApp()
    app.run()
